<a href="https://colab.research.google.com/github/dcolinmorgan/LTCOPDsex/blob/master/v4_u_2xCOPD_gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install s3fs
import os,time
# import s3fs
import pandas as pd
import numpy as np
from psutil import *
import matplotlib.pyplot as plt
import gc
import random
from scipy import stats
import seaborn as sns
from IPython.display import Image
from scipy.stats import zscore
timestr = time.strftime("%Y%m%d")
import requests 
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# data=np.load('/content/drive/MyDrive/LARGE_PANDA/LTRCcase.npy')
# plt.hist(data.flatten())

In [3]:
# !git clone https://github.com/netZoo/netZooPy.git
!git clone --single-branch --branch devel https://github.com/netZoo/netZooPy.git
os.chdir('netZooPy')
!pip install -e ./

Cloning into 'netZooPy'...
remote: Enumerating objects: 4599, done.
remote: Counting objects: 100% (392/392), done.
remote: Compressing objects: 100% (231/231), done.
remote: Total 4599 (delta 251), reused 261 (delta 146), pack-reused 4207
Receiving objects: 100% (4599/4599), 123.82 MiB | 26.58 MiB/s, done.
Resolving deltas: 100% (2249/2249), done.
Obtaining file:///content/netZooPy
     |████████████████████████████████| 3.2MB 7.8MB/s 
  Running setup.py develop for netZooPy


In [4]:
import netZooPy
from netZooPy.panda import Panda
from netZooPy.lioness import Lioness
# from netZooPy import condor
os.chdir('..')

In [5]:
!pip install GEOparse
# !pip install pycombat

     |████████████████████████████████| 286kB 8.1MB/s 
  Created wheel for GEOparse: filename=GEOparse-2.0.3-cp37-none-any.whl size=29066 sha256=5237fbdf87bb4944151565a789f28c2d1ff59adfa5057eb5a0fde0e58a52a916
  Stored in directory: /root/.cache/pip/wheels/c6/5e/b4/7109a7b8e8046bd8586349a6ac1a76ef8190d12a61a7b4cf0d
Successfully built GEOparse


In [ ]:
import GEOparse
gse = GEOparse.get_GEO(geo="GSE76925", destdir="./")

14-Jul-2021 11:01:36 DEBUG utils - Directory ./ already exists. Skipping.
14-Jul-2021 11:01:36 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE76nnn/GSE76925/soft/GSE76925_family.soft.gz to ./GSE76925_family.soft.gz
100%|██████████| 70.3M/70.3M [00:02<00:00, 30.0MB/s]
14-Jul-2021 11:01:40 DEBUG downloader - Size validation passed
14-Jul-2021 11:01:40 DEBUG downloader - Moving /tmp/tmpuqe4_e76 to /content/GSE76925_family.soft.gz
14-Jul-2021 11:01:40 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE76nnn/GSE76925/soft/GSE76925_family.soft.gz
14-Jul-2021 11:01:40 INFO GEOparse - Parsing ./GSE76925_family.soft.gz: 
14-Jul-2021 11:01:40 DEBUG GEOparse - DATABASE: GeoMiame
14-Jul-2021 11:01:40 DEBUG GEOparse - SERIES: GSE76925
14-Jul-2021 11:01:40 DEBUG GEOparse - PLATFORM: GPL10558
14-Jul-2021 11:01:42 DEBUG GEOparse - SAMPLE: GSM2040792
14-Jul-2021 11:01:42 DEBUG GEOparse - SAMPLE: GSM2040793
14-Jul-2021 11:01:42 DEBUG GEOparse - SAMP

In [ ]:
ww=pd.unique(gse.gpls['GPL10558'].table['Chromosome'])

In [ ]:
gene2chr=gse.gpls['GPL10558'].table[['ILMN_Gene','Chromosome']]

In [ ]:
# data1=pd.read_csv('drive/My Drive/Colab Notebooks/GSE76925.txt',sep='\t',index_col=0)

In [ ]:
# largeCOPD=pd.read_csv('drive/My Drive/Colab Notebooks/LARGE_COPD_crossLTCOPD.txt',sep='\t',index_col=0)

###this data is same as clean expression per Jarrett

In [ ]:
del data1['loc']
plot=pd.melt(data1)['value']
# plot=np.array(plot.astype(float))
plt.hist(plot,bins=26)

In [ ]:
Image(filename='drive/My Drive/Colab Notebooks/clean_LTCOPD.png',width=500)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
cc=plt.hist(np.log(data1.values.flatten()),bins=100)

##test merged lung tissue datasets in PCA and heirarchical clustering

In [ ]:
X=largeCOPD#.merge(data1,left_index=True, right_index=True)
X=X.T
y=(pd.DataFrame(np.zeros(largeCOPD.shape[1]))).astype('int').to_numpy().flatten()
# from sklearn.cluster import AgglomerativeClustering
X.fillna(0, inplace=True)
# model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
from sklearn.decomposition import PCA
lw = 2
pca = PCA(n_components=lw)
X_r=pca.fit(X).transform(X)
# model = model.fit(data1)
from sklearn import datasets
iris = datasets.load_iris()
target_names = ['large','small']#iris.target_names

plt.figure(figsize=(20, 4))

colors = ['turquoise', 'darkorange']


for color, i, target_name in zip(colors, [0, 1], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of LT-RC datasets')

In [ ]:
X=data1#,left_index=True, right_index=True)
X=X.T
y=(pd.DataFrame(np.zeros(data1.shape[1]))).astype('int').to_numpy().flatten()
# from sklearn.cluster import AgglomerativeClustering
X.fillna(0, inplace=True)
# model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
from sklearn.decomposition import PCA
lw = 2
pca = PCA(n_components=lw)
X_r=pca.fit(X).transform(X)
# model = model.fit(data1)
from sklearn import datasets
iris = datasets.load_iris()
target_names = ['large','small']#iris.target_names

plt.figure(figsize=(20, 4))

colors = ['turquoise', 'darkorange']


for color, i, target_name in zip(colors, [0, 1], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.8, lw=lw,
                label=target_name)
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of LT-RC datasets')

In [ ]:
X=largeCOPD.merge(data1,left_index=True, right_index=True)
X=X.T
y=(pd.DataFrame(np.zeros(largeCOPD.shape[1])).append(pd.DataFrame(np.ones(data1.shape[1])))).astype('int').to_numpy().flatten()
# from sklearn.cluster import AgglomerativeClustering
X.fillna(0, inplace=True)
# model = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
from sklearn.decomposition import PCA
lw = 2
pca = PCA(n_components=lw)
X_r=pca.fit(X).transform(X)
# model = model.fit(data1)
from sklearn import datasets
iris = datasets.load_iris()
target_names = ['large','small']#iris.target_names

plt.figure(figsize=(20, 20))

colors = ['turquoise', 'darkorange']


for color, i, target_name in zip(colors, [0, 1], target_names):
    plt.scatter(X_r[y == i, 0], X_r[y == i, 1], color=color, alpha=.5, lw=lw,
                label=target_name, )
plt.legend(loc='best', shadow=False, scatterpoints=1)
plt.title('PCA of LT-RC and LT-COPD datasets')

In [ ]:
from scipy.cluster.hierarchy import dendrogram
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack([model.children_, model.distances_,
                                      counts]).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)


In [ ]:
plt.figure(figsize=(20, 4))
plt.title('Hierarchical Clustering Dendrogram')
# plot the top three levels of the dendrogram
plot_dendrogram(model, truncate_mode='level', p=3,)
plt.xlabel("Number of points in node (or index of point if no parenthesis).")
plt.show()

In [ ]:
plt.xkcd();
from scipy import stats, spatial, cluster
Z = cluster.hierarchy.linkage(cc, "complete")

In [ ]:
plt.style.use('classic')
plt.figure(figsize=(70, 4))
plt.title("Here's a dendrogram of our hierarchical clustering")
cluster.hierarchy.dendrogram(Z);

#create and run PANDA

In [ ]:
gse.pivot_samples


In [ ]:
expression1 = gse.pivot_samples('VALUE')
gpl = gse.gpls[next(iter(gse.gpls))]
data = gse.pivot_samples(values="VALUE")#, gpl=gpl, annotation_column="Symbol")
pheno=gse.phenotype_data.rename(columns={'characteristics_ch1.0.age': 'age',"characteristics_ch1.1.Sex":"sex","characteristics_ch1.3.bmi":"bmi","characteristics_ch1.6.fev1.pp":"fev1","characteristics_ch1.4.packyears":"PY"})
pheno[['age',"sex","bmi","fev1","PY"]].to_csv('pheno_data.txt',sep='\t',index=True,header=True)

names=pd.DataFrame([gse.gpls['GPL10558'].table['ID'],gse.gpls['GPL10558'].table['ILMN_Gene']]).transpose()
data1=pd.merge(expression1,names,left_index=True,right_on='ID')

data1['loc']=gse.gpls['GPL10558'].table['Chromosome']

data1.index=data1['ILMN_Gene']
data1=data1.dropna(how='any')
del data1['ID'], data1['ILMN_Gene']

# data1.to_csv('drive/My Drive/Colab Notebooks/GSE76925.txt',sep='\t')

In [ ]:
jeff=(data1)
cc=jeff[jeff['loc']=='Y'].index

In [ ]:
ltrc_genes=pd.read_csv('drive/My Drive/Colab Notebooks/LTRC_genes_chr.txt',sep='\t')
data2=data1.merge(ltrc_genes,left_index=True,right_on='gene')

In [ ]:
pd.unique(data2.gene).shape

In [ ]:
dd=pd.unique(data2['gene'][data2['chr']=='chrY'].tolist())

In [ ]:
data2.index=data2.gene
data2=data2.groupby(data2.index).mean()
del data2['GSM2040941'],data2['GSM2040942'] ##remove rather than combat adjusting, only two samples from seperate batch
# combat.fit(Y=data2, b=b, X=X, C=C)

In [ ]:
pd.DataFrame(data2.index).to_csv('drive/My Drive/Colab Notebooks/LTCOPD_genes.txt',index=False,header=False,sep='\t')

In [ ]:
# data3=data1data1[data1['loc']!='Y']

# del data2['loc'], data2['ensg'], data2['gene'], data2['chr']

data2.columns=[gse.phenotype_data['characteristics_ch1.1.Sex'][:-2]+gse.phenotype_data['characteristics_ch1.5.copd'][:-2]]
data2.loc[:,'Fcase'].to_csv('drive/My Drive/harvard/Fcase.txt',sep='\t',index=True,header=False)
data2.loc[:,'Fcont'].to_csv('drive/My Drive/harvard/Fcont.txt',sep='\t',index=True,header=False)
print([data2.loc[:,'Fcase'].shape,data2.loc[:,'Fcont'].shape])

# data4=data1 #pd.DataFrame(np.copy(data1)) #data4[data4['loc']!='Y']
# del data4['loc']
# data2.columns=[gse.phenotype_data['characteristics_ch1.1.Sex']+gse.phenotype_data['characteristics_ch1.5.copd']]
data2.loc[:,'Mcase'].to_csv('drive/My Drive/harvard/Mcase.txt',sep='\t',index=True,header=False)
data2.loc[:,'Mcont'].to_csv('drive/My Drive/harvard/Mcont.txt',sep='\t',index=True,header=False)
print([data2.loc[:,'Mcase'].shape,data2.loc[:,'Mcont'].shape])

In [ ]:
# LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/ppi/ppi_complete.txt',sep='\t',header=None)
# LCL_ppi.to_csv('drive/My Drive/Colab Notebooks/ppi_complete.txt',sep='\t',index=False,header=False)

coding_LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/motifs/Hugo_motifCellLine.txt',sep='\t',header=None)
coding_LCL_motif=coding_LCL_motif.reset_index().rename(columns={0:'source',1:'target', 2:'weight'})
del coding_LCL_motif['index']
# coding_LCL_motif.to_csv('drive/My Drive/Colab Notebooks/coding_LCL_motif.txt',sep='\t',index=False,header=False)
# !curl -O https://cmapreg.s3.us-east-2.amazonaws.com/regMatBin1thr005.csv
motif_dataBin1=pd.read_csv('https://cmapreg.s3.us-east-2.amazonaws.com/regMatBin1thr005.csv',sep=',',header=0,index_col=0)
full_motif=motif_dataBin1.stack().reset_index().rename(columns={'level_0':'source','level_1':'target', 0:'weight'})
# full_motif.to_csv('drive/My Drive/Colab Notebooks/full_LCL_motif.txt',sep='\t',index=False,header=False)

full_motif=full_motif.merge(coding_LCL_motif,on=['source','target'],how='left')
full_motif.loc[full_motif["weight_y"].isnull(),'weight_y'] = full_motif['weight_x']
full_motif=full_motif[['source','target','weight_y']]
full_motif=full_motif.rename(columns={'source':'source','target':'target', 'weight_y':'weight'})
full_motif = full_motif[~full_motif['source'].isin(coding_LCL_motif['source'])]
full_motif[['source','target','weight']].to_csv('drive/My Drive/Colab Notebooks/coding_full_LCL_motif.txt',sep='\t',index=False,header=False)

motif_data='drive/My Drive/Colab Notebooks/coding_full_LCL_motif.txt' ##coding_LCL_motif.txt
ppi_data='drive/My Drive/Colab Notebooks/ppi_complete.txt'
F_motif_data='drive/My Drive/Colab Notebooks/F_coding_full_LCL_motif.txt' ##F_coding_LCL_motif.txt

In [ ]:
# full_motif=full_motif[['source','target','weight']]
full_motif.columns=['TF','gene','W']
aa=full_motif[~full_motif['TF'].isin(cc)]
bb=full_motif[full_motif['TF'].isin(cc)]
bb['W']=0
full_motif=aa.append(bb)

aa=full_motif[~full_motif['gene'].isin(cc)]
bb=full_motif[full_motif['gene'].isin(cc)]
bb['W']=0
full_motif=aa.append(bb)
full_motif.to_csv('drive/My Drive/Colab Notebooks/F_coding_full_LCL_motif.txt',sep='\t',index=False,header=False)
# F_motif_data='drive/My Drive/Colab Notebooks/F_full_motif.txt'

In [ ]:
# panda_obj1 = Panda('drive/My Drive/harvard/Fcase.txt', F_motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
# np.save('drive/My Drive/Colab Notebooks/model_output/Panda_F0case_full.npy',panda_obj1.export_panda_results)
# del panda_obj1
panda_obj2 = Panda('drive/My Drive/harvard/Mcase.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/Panda_Mcase_full.npy',panda_obj2.export_panda_results)
del panda_obj2
panda_obj3 = Panda('drive/My Drive/harvard/Fcont.txt', F_motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/Panda_F0cont_full.npy',panda_obj3.export_panda_results)
del panda_obj3
panda_obj4 = Panda('drive/My Drive/harvard/Mcont.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/Panda_Mcont_full.npy',panda_obj4.export_panda_results)
del panda_obj4

##Load Panda nets

In [ ]:
# Panda_Fcase=np.load('drive/My Drive/Colab Notebooks dcm/Panda_Fcase.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# Panda_Fcont=np.load('drive/My Drive/Colab Notebooks dcm/Panda_Fcont.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# Panda_Fcont=pd.DataFrame(Panda_Fcont)
# Panda_Fcont.columns=['tf','gene','motif','force']
# Panda_Fcase=pd.DataFrame(Panda_Fcase)
# Panda_Fcase.columns=['tf','gene','motif','force']

Panda_Mcase=np.load('drive/My Drive/Colab Notebooks/model_output/Panda_Mcase_full.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_Mcont=np.load('drive/My Drive/Colab Notebooks/model_output/Panda_Mcont_full.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_Mcont=pd.DataFrame(Panda_Mcont)
Panda_Mcont.columns=['tf','gene','motif','force']
Panda_Mcase=pd.DataFrame(Panda_Mcase)
Panda_Mcase.columns=['tf','gene','motif','force']
## process F and M one at a time now to save memory

Panda_FcaseFULL=np.load('drive/My Drive/Colab Notebooks/model_output/Panda_F0case_full.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_FcontFULL=np.load('drive/My Drive/Colab Notebooks/model_output/Panda_F0cont_full.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_FcontFULL=pd.DataFrame(Panda_FcontFULL)
Panda_FcontFULL.columns=['tf','gene','motif','force']
Panda_FcaseFULL=pd.DataFrame(Panda_FcaseFULL)
Panda_FcaseFULL.columns=['tf','gene','motif','force']


##TF and gene specific zscore diff F



In [ ]:
##control minus case
from scipy.stats import zscore
# Panda_nullA.panda_results = pd.DataFrame(Panda_nullA.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results3 = Panda_FcontFULL.sort_values(by=['force'], ascending=False)
# subset_panda_results3['merge']=subset_panda_results3.tf+'-'+subset_panda_results3.gene

# Panda_nullB.panda_results = pd.DataFrame(Panda_nullB.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results1 = Panda_FcaseFULL.sort_values(by=['force'], ascending=False)
# subset_panda_results1['merge']=subset_panda_results1.tf+'-'+subset_panda_results1.gene
del subset_panda_results3['motif'],subset_panda_results1['motif']
TFsum_3=subset_panda_results3.groupby(['tf']).agg({'force':sum})
TFsum_1=subset_panda_results1.groupby(['tf']).agg({'force':sum})
TFsum_1['force']=zscore(TFsum_1['force'])
TFsum_3['force']=zscore(TFsum_3['force']) ##reduces range from 1500 to 3

geneSum_3=subset_panda_results3.groupby(['gene']).agg({'force':sum})
geneSum_1=subset_panda_results1.groupby(['gene']).agg({'force':sum})
geneSum_3['force']=zscore(geneSum_3['force'])
geneSum_1['force']=zscore(geneSum_1['force'])

mmm=TFsum_3.merge(TFsum_1, left_on=['tf'], right_on=['tf'])
mmmm=geneSum_3.merge(geneSum_1, left_on=['gene'], right_on=['gene'])

del [[subset_panda_results3, subset_panda_results1,Panda_FcontFULL,Panda_FcaseFULL]]
gc.collect()
subset_panda_results3=pd.DataFrame()
subset_panda_results1=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()


mmm['diff']=mmm.force_x-mmm.force_y
mmm = mmm.sort_values(by ='diff' )


mmmm['diff']=mmmm.force_x-mmmm.force_y
mmmm = mmmm.sort_values(by ='diff' )
mmm.to_csv('drive/My Drive/TF_zscore_F_u.txt',sep='\t')
mmmm.to_csv('drive/My Drive/gene_zscore_F_u.txt',sep='\t')

##TF and gene specific zscore diff M

In [ ]:
# Panda_nullA.panda_results = pd.DataFrame(Panda_nullA.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results3 = Panda_Mcont.sort_values(by=['force'], ascending=False)
# subset_panda_results3['merge']=subset_panda_results3.tf+'-'+subset_panda_results3.gene

# Panda_nullB.panda_results = pd.DataFrame(Panda_nullB.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results1 = Panda_Mcase.sort_values(by=['force'], ascending=False)
# subset_panda_results1['merge']=subset_panda_results1.tf+'-'+subset_panda_results1.gene
del subset_panda_results3['motif'],subset_panda_results1['motif']
TFsum_3=subset_panda_results3.groupby(['tf']).agg({'force':sum})
TFsum_1=subset_panda_results1.groupby(['tf']).agg({'force':sum})
TFsum_1['force']=zscore(TFsum_1['force'])
TFsum_3['force']=zscore(TFsum_3['force'])

geneSum_3=subset_panda_results3.groupby(['gene']).agg({'force':sum})
geneSum_1=subset_panda_results1.groupby(['gene']).agg({'force':sum})
geneSum_3['force']=zscore(geneSum_3['force'])
geneSum_1['force']=zscore(geneSum_1['force'])

nnn=TFsum_3.merge(TFsum_1, left_on=['tf'], right_on=['tf'])
nnnn=geneSum_3.merge(geneSum_1, left_on=['gene'], right_on=['gene'])

del [[subset_panda_results3, subset_panda_results1,Panda_Mcont,Panda_Mcase]]
gc.collect()
subset_panda_results3=pd.DataFrame()
subset_panda_results1=pd.DataFrame()
Panda_Mcont=pd.DataFrame()
Panda_Mcase=pd.DataFrame()

nnn['diff']=nnn.force_x-nnn.force_y
nnn = nnn.sort_values(by ='diff' )

nnnn['diff']=nnnn.force_x-nnnn.force_y
nnnn = nnnn.sort_values(by ='diff' )
nnn.to_csv('drive/My Drive/TF_zscore_M_u.txt',sep='\t')
nnnn.to_csv('drive/My Drive/gene_zscore_M_u.txt',sep='\t')

In [ ]:
sex_tf_diff=mmm.merge(nnn,on='tf')
a=plt.hist(sex_tf_diff['diff_x'],bins=100,label='male',alpha=.75)
b=plt.hist(sex_tf_diff['diff_y'],bins=100,label='female',alpha=.5)
plt.title('Sex-specific Ca-Co TF degree diff')
plt.legend(loc="best")

In [ ]:
[sex_tf_diff[(sex_tf_diff['diff_y'])>.4].index.values,sex_tf_diff[(sex_tf_diff['diff_y'])<-0.5].index.values] #,sex_tf_diff[np.abs(sex_tf_diff['diff_y'])>.4].tf]

In [ ]:
sex_gene_diff=mmmm.merge(nnnn,on='gene')
a=plt.hist(sex_gene_diff['diff_x'],bins=100,label='male')
b=plt.hist(sex_gene_diff['diff_y'],bins=100,label='female')
plt.title('Sex-specific Ca-Co gene degree diff')
plt.legend(loc="best")

In [ ]:
[sex_gene_diff[(sex_gene_diff['diff_y'])>1].index.values,sex_gene_diff[(sex_gene_diff['diff_y'])<-1].index.values] #,sex_tf_diff[np.abs(sex_tf_diff['diff_y'])>.4].tf]

#Run larger COPD panda nets

In [ ]:
# LTRC_exp=pd.read_csv('drive/MyDrive/harvard/LARGE_COPD/LTRC_XXX_060521.txt',sep='\t',index_col=0)
# LTRC_exp_key=pd.read_csv('drive/MyDrive/harvard/LARGE_COPD/LTRC_XXXkey_060521.txt',sep='\t',index_col=0)
# data=np.multiply(LTRC_exp,LTRC_exp_key)
# LTRC_exp=pd.read_csv('drive/MyDrive/harvard/LARGE_COPD/LTRC_qsmooth_combat.txt',sep='\t',index_col=0)
LTRC_exp=pd.read_csv('drive/MyDrive/harvard/LARGE_COPD/LTRC_qs_log2_removebatch.txt',sep='\t',index_col=0)

ltrc_genes=pd.read_csv('drive/My Drive/Colab Notebooks/LTRC_genes_chr.txt',sep='\t')
LTRC_exp.index=LTRC_exp.index.str.split('.').str[0].tolist()
data1=LTRC_exp.merge(ltrc_genes,left_index=True,right_on='ensg')
data1.index=data1['gene']
del data1['ensg'], data1['chr']
data1=data1.groupby(data1.index).mean()

In [ ]:
LTCOPD_genes=pd.read_csv('drive/My Drive/Colab Notebooks/LTCOPD_genes.txt',index_col=None,header=None,sep='\t')
# data1=np.log2(data1.dropna(how='all')+1)
data1=data1.merge(LTCOPD_genes,left_index=True,right_on=0)
data1.index=data1[0]
data1.replace([np.inf, -np.inf], np.nan,inplace=True)
data1.replace(np.nan, 0,inplace=True)

In [ ]:
# gene2chr[gene2chr['ILMN_Gene']=='GATA1']

In [ ]:
# data1[data1.index=='GATA1'] 

In [ ]:
# data1.to_csv('LTRC_qs_com_exp.txt',sep='\t',index=True,header=True)

In [ ]:
del data1[0]
FC=data1.iloc[:,data1.columns.str.contains('_0_1')]
FNC=data1.iloc[:,data1.columns.str.contains('_0_0')]
MC=data1.iloc[:,data1.columns.str.contains('_1_1')]
MNC=data1.iloc[:,data1.columns.str.contains('_1_0')]
print([FC.shape,FNC.shape,MC.shape,MNC.shape])
FC.to_csv('drive/My Drive/harvard/LARGE_COPD/FC_exp.txt',sep='\t',index=True,header=False)
FNC.to_csv('drive/My Drive/harvard/LARGE_COPD/FNC_exp.txt',sep='\t',index=True,header=False)
MC.to_csv('drive/My Drive/harvard/LARGE_COPD/MC_exp.txt',sep='\t',index=True,header=False)
MNC.to_csv('drive/My Drive/harvard/LARGE_COPD/MNC_exp.txt',sep='\t',index=True,header=False)

In [ ]:
# LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/ppi/ppi_complete.txt',sep='\t',header=None)
# LCL_ppi.to_csv('drive/My Drive/Colab Notebooks/ppi_complete.txt',sep='\t',index=False,header=False)

# coding_LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/motifs/Hugo_motifCellLine.txt',sep='\t',header=None)
# coding_LCL_motif.to_csv('drive/My Drive/Colab Notebooks/coding_LCL_motif.txt',sep='\t',index=False,header=False)

# drive/My Drive/Colab Notebooks/coding_LCL_motif.txt
motif_data='drive/My Drive/Colab Notebooks/coding_full_LCL_motif.txt'
ppi_data='drive/My Drive/Colab Notebooks/ppi_complete.txt'
F_motif_data='drive/My Drive/Colab Notebooks/F_coding_full_LCL_motif.txt'

In [ ]:
panda_obj1 = Panda('drive/My Drive/harvard/LARGE_COPD/FC_exp.txt', F_motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/FC_LTRC_rmb1.npy',np.round(panda_obj1.export_panda_results,decimals=3))
del panda_obj1
panda_obj2 = Panda('drive/My Drive/harvard/LARGE_COPD/MC_exp.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/MC_LTRC_rmb1.npy',np.round(panda_obj2.export_panda_results,decimals=3))
del panda_obj2
panda_obj3 = Panda('drive/My Drive/harvard/LARGE_COPD/FNC_exp.txt', F_motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/FNC_LTRC_rmb1.npy',np.round(panda_obj3.export_panda_results,decimals=3))
del panda_obj3
panda_obj4 = Panda('drive/My Drive/harvard/LARGE_COPD/MNC_exp.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/MNC_LTRC_rmb1.npy',np.round(panda_obj4.export_panda_results,decimals=3))
del panda_obj4
# panda_obj5 = Panda('FcaseFULL.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
# np.save('drive/My Drive/Panda_FcaseFULL.npy',panda_obj5.export_panda_results)
# del panda_obj5
# panda_obj6 = Panda('FcontFULL.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
# np.save('drive/My Drive/Panda_FcontFULL.npy',panda_obj6.export_panda_results)
# del panda_obj6


##MALE

In [ ]:
from scipy.stats import zscore

In [ ]:
MC=np.load('drive/My Drive/Colab Notebooks/model_output/MC_LTRC_rmb1.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
MNC=np.load('drive/My Drive/Colab Notebooks/model_output/MNC_LTRC_rmb1.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
MNC=pd.DataFrame(MNC)
MNC.columns=['tf','gene','motif','force']
MC=pd.DataFrame(MC)
MC.columns=['tf','gene','motif','force']

In [ ]:
# len(pd.unique(MC.gene))#
# MC['gene']=='TSPAN6']

In [ ]:
# MC_chr=MC.merge(gene2chr,left_on='tf',right_on='ILMN_Gene')
# pd.unique(MC_chr[MC_chr['Chromosome']=='X'].tf)

In [ ]:
# MC_chr[MC_chr['gene']=='GATA1']
# coding_LCL_motif[coding_LCL_motif[0]=='GATA1']

In [ ]:
## case minus control
TFsum_3=MC.groupby(['tf']).agg({'force':sum})
TFsum_1=MNC.groupby(['tf']).agg({'force':sum})
TFsum_1['force']=zscore(TFsum_1['force'])
TFsum_3['force']=zscore(TFsum_3['force']) ##reduces range from 1500 to 3

geneSum_3=MC.groupby(['gene']).agg({'force':sum})
geneSum_1=MNC.groupby(['gene']).agg({'force':sum})
geneSum_3['force']=zscore(geneSum_3['force'])
geneSum_1['force']=zscore(geneSum_1['force']) ## perhaps try  qnorm rather than zscore

mmm=TFsum_3.merge(TFsum_1, left_on=['tf'], right_on=['tf'])
mmmm=geneSum_3.merge(geneSum_1, left_on=['gene'], right_on=['gene'])

# del [[MC, MNC]]
# gc.collect()
MC=pd.DataFrame()
MNC=pd.DataFrame()
# Panda_FcontFULL=pd.DataFrame()
# Panda_FcontFULL=pd.DataFrame()


mmm['diff']=mmm.force_x-mmm.force_y
mmm = mmm.sort_values(by ='diff' )


mmmm['diff']=mmmm.force_x-mmmm.force_y
mmmm = mmmm.sort_values(by ='diff' )
mmm.to_csv('drive/My Drive/TF_zscore_M_large_u.txt',sep='\t')
mmmm.to_csv('drive/My Drive/gene_zscore_M_large_u.txt',sep='\t')

##FEMALE

In [ ]:
FC=np.load('drive/My Drive/Colab Notebooks/model_output/FC_LTRC_rmb1.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
FNC=np.load('drive/My Drive/Colab Notebooks/model_output/FNC_LTRC_rmb1.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
FNC=pd.DataFrame(FNC)
FNC.columns=['tf','gene','motif','force']
FC=pd.DataFrame(FC)
FC.columns=['tf','gene','motif','force']

In [ ]:
TFsum_4=FC.groupby(['tf']).agg({'force':sum})
TFsum_2=FNC.groupby(['tf']).agg({'force':sum})
TFsum_2['force']=zscore(TFsum_2['force'])
TFsum_4['force']=zscore(TFsum_4['force']) ##reduces range from 2500 to 4

geneSum_4=FC.groupby(['gene']).agg({'force':sum})
geneSum_2=FNC.groupby(['gene']).agg({'force':sum})
geneSum_4['force']=zscore(geneSum_4['force'])
geneSum_2['force']=zscore(geneSum_2['force'])

nnn=TFsum_4.merge(TFsum_2, left_on=['tf'], right_on=['tf'])
nnnn=geneSum_4.merge(geneSum_2, left_on=['gene'], right_on=['gene'])

# del [[FC, FNC]]
# gc.collect()
FC=pd.DataFrame()
FNC=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()


nnn['diff']=nnn.force_x-nnn.force_y
nnn = nnn.sort_values(by ='diff' )


nnnn['diff']=mmmm.force_x-nnnn.force_y
nnnn = nnnn.sort_values(by ='diff' )
nnn.to_csv('drive/My Drive/TF_zscore_F_large_u.txt',sep='\t')
nnnn.to_csv('drive/My Drive/gene_zscore_F_large_u.txt',sep='\t')

In [ ]:
# ## subtract link weights between case and control, then zscore then take degree

# MC=np.load('drive/My Drive/Colab Notebooks/model_output/MC_LTRC.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# MNC=np.load('drive/My Drive/Colab Notebooks/model_output/MNC_LTRC.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# MNC=pd.DataFrame(MNC)
# MNC.columns=['tf','gene','motif','force']
# MC=pd.DataFrame(MC)
# MC.columns=['tf','gene','motif','force']

# FC=np.load('drive/My Drive/Colab Notebooks/model_output/FC_LTRC.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# FNC=np.load('drive/My Drive/Colab Notebooks/model_output/FNC_LTRC.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# FNC=pd.DataFrame(FNC)
# FNC.columns=['tf','gene','motif','force']
# FC=pd.DataFrame(FC)
# FC.columns=['tf','gene','motif','force']

# FC['Ca-Co_diff']=zscore(pd.to_numeric(FC.force-FNC.force))
# FeAtf=FC.groupby(['tf']).agg({'force':sum})
# FeAg=FC.groupby(['gene']).agg({'Ca-Co_diff':sum})
# MC['Ca-Co_diff']=zscore(pd.to_numeric(MC.force-MNC.force))
# MaAtf=MC.groupby(['tf']).agg({'force':sum})
# MaAg=MC.groupby(['tf']).agg({'Ca-Co_diff':sum})

# # FC['Ca-Co_diff']=zscore(pd.to_numeric(FC.force-FNC.force))
# # FeDiffA=FC.groupby(['tf']).agg({'Ca-Co_diff':sum})
# # MC['Ca-Co_diff']=zscore(pd.to_numeric(MC.force-MNC.force))#
# # MaDiffA=MC.groupby(['tf']).agg({'Ca-Co_diff':sum})

# del FC, FNC, MC, MNC

# Panda_Mcase=np.load('drive/My Drive/Colab Notebooks/model_output/MC_LTCOPD.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# Panda_Mcont=np.load('drive/My Drive/Colab Notebooks/model_output/MNC_LTCOPD.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# Panda_Mcont=pd.DataFrame(Panda_Mcont)
# Panda_Mcont.columns=['tf','gene','motif','force']
# Panda_Mcase=pd.DataFrame(Panda_Mcase)
# Panda_Mcase.columns=['tf','gene','motif','force']

# Panda_FcaseFULL=np.load('drive/My Drive/Colab Notebooks/model_output/FC_LTCOPD.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# Panda_FcontFULL=np.load('drive/My Drive/Colab Notebooks/model_output/FNC_LTCOPD.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
# Panda_FcontFULL=pd.DataFrame(Panda_FcontFULL)
# Panda_FcontFULL.columns=['tf','gene','motif','force']
# Panda_FcaseFULL=pd.DataFrame(Panda_FcaseFULL)
# Panda_FcaseFULL.columns=['tf','gene','motif','force']

# Panda_FcaseFULL['Ca-Co_diff']=zscore(pd.to_numeric(Panda_FcaseFULL.force-Panda_FcontFULL.force))
# FeBtf=Panda_FcaseFULL.groupby(['tf']).agg({'force':sum})
# FeBg=Panda_FcaseFULL.groupby(['gene']).agg({'Ca-Co_diff':sum})
# Panda_Mcase['Ca-Co_diff']=zscore(pd.to_numeric(Panda_Mcase.force-Panda_Mcont.force))
# MaBtf=Panda_Mcase.groupby(['tf']).agg({'force':sum})
# MaBg=Panda_Mcase.groupby(['gene']).agg({'Ca-Co_diff':sum})

# # Panda_FcaseFULL['Ca-Co_diff']=zscore(pd.to_numeric(Panda_FcaseFULL.force-Panda_FcontFULL.force))
# # FeDiffB=Panda_FcaseFULL.groupby(['tf']).agg({'Ca-Co_diff':sum})
# # Panda_Mcase['Ca-Co_diff']=zscore(pd.to_numeric(Panda_Mcase.force-Panda_Mcont.force))#
# # MaDiffB=Panda_Mcase.groupby(['tf']).agg({'Ca-Co_diff':sum})

# del Panda_FcaseFULL, Panda_FcontFULL, Panda_Mcase, Panda_Mcont

# plt.figure(figsize=(8, 5))

# p1=plt.scatter(MaBtf,MaAtf,alpha=.5) ## minus because one is case-control, other control-case!!
# p2=plt.scatter(FeBtf,FeAtf,alpha=.25)
# plt.ylabel('LTRC case-control TF targeting diff')
# plt.xlabel('LTCOPD case-control TF targeting diff')
# plt.legend((p1, p2),('Male','Female'))
# plt.savefig("fig.png",dpi=300,bbox_inches = "tight")

In [ ]:
# rhoM, pvalM = np.corrcoef(MaBtf['Ca-Co_diff'], MaAtf['Ca-Co_diff'])
# # pvalM
# rhoF, pvalF = np.corrcoef(FeBtf['Ca-Co_diff'], FeAtf['Ca-Co_diff'])
# # pvalF
# print([rhoM[1],rhoF[1]])

# rhoM, pvalM = stats.spearmanr(MaBtf['Ca-Co_diff'], MaAtf['Ca-Co_diff'])
# # pvalM
# rhoF, pvalF = stats.spearmanr(FeBtf['Ca-Co_diff'], FeAtf['Ca-Co_diff'])
# # pvalF
# print([rhoM,rhoF])

In [ ]:
# Ma=MaBg.merge(MaAg,left_index=True,right_index=True)
# Fe=FeBg.merge(FeAg,left_index=True,right_index=True)
# p1=plt.scatter(Ma['Ca-Co_diff_x'],Ma['Ca-Co_diff_y'],alpha=.5) ## minus because one is case-control, other control-case!!
# p2=plt.scatter(Fe['Ca-Co_diff_x'],Fe['Ca-Co_diff_y'],alpha=.25)
# plt.ylabel('LTRC case-control TF targeting diff')
# plt.xlabel('LTCOPD case-control TF targeting diff')
# plt.legend((p1, p2),('Male','Female'))
# plt.savefig("fig.png",dpi=300,bbox_inches = "tight")

In [ ]:
sex_tf_diff=mmm.merge(nnn,on='tf')
a=plt.hist(sex_tf_diff['diff_x'],bins=100,label='male')
b=plt.hist(sex_tf_diff['diff_y'],bins=100,label='female')
plt.title('Sex-specific Ca-Co TF degree diff')
plt.legend(loc="best")

In [ ]:
[sex_tf_diff[(sex_tf_diff['diff_y'])>.15].index.values,sex_tf_diff[(sex_tf_diff['diff_y'])<-0.15].index.values] #,sex_tf_diff[np.abs(sex_tf_diff['diff_y'])>.4].tf]

In [ ]:
sex_gene_diff=mmmm.merge(nnnn,on='gene')
a=plt.hist(sex_gene_diff['diff_x'],bins=100,label='male')
b=plt.hist(sex_gene_diff['diff_y'],bins=100,label='female')
plt.title('Sex-specific Ca-Co gene degree diff')
# plt.xlim(.1,-0.1)
plt.legend(loc="best")

In [ ]:
[sex_gene_diff[(sex_gene_diff['diff_y'])>1].index.values,sex_gene_diff[(sex_gene_diff['diff_y'])<-1].index.values] #,sex_tf_diff[np.abs(sex_tf_diff['diff_y'])>.4].tf]

In [ ]:
ff=pd.read_csv('drive/My Drive/TF_zscore_F_large_u.txt',sep='\t',header=0)
fff=pd.read_csv('drive/My Drive/gene_zscore_F_large_u.txt',sep='\t',header=0)
mm=pd.read_csv('drive/My Drive/TF_zscore_M_large_u.txt',sep='\t',header=0)
mmm=pd.read_csv('drive/My Drive/gene_zscore_M_large_u.txt',sep='\t',header=0)
sex_tf_diff=ff.merge(mm,on='tf')

# a=plt.hist(sex_tf_diff['diff_x'],bins=100,label='female')
# b=plt.hist(sex_tf_diff['diff_y'],bins=100,label='male')
# plt.title('LTRC Sex-specific Ca-Co TF degree diff')
# plt.legend(loc="best")

# [sex_tf_diff[(sex_tf_diff['diff_y'])>.1].TF.values,sex_tf_diff[(sex_tf_diff['diff_y'])<-0.1].TF.values] #,sex_tf_diff[np.abs(sex_tf_diff['diff_y'])>.4].tf]

sex_tf_diff.columns=['TF','F_case','F_cont','Fdiff','M_case','M_cont','Mdiff']
sex_tf_diff.to_csv('drive/My Drive/TF_sex_diff.txt',sep='\t',index=0)


sex_gene_diff=fff.merge(mmm,on='gene')
sex_gene_diff.columns=['gene','F_case','F_cont','Fdiff','M_case','M_cont','Mdiff']
sex_gene_diff.to_csv('drive/My Drive/gene_sex_diff.txt',sep='\t',index=0)


##Compare All

In [ ]:
ff=pd.read_csv('drive/My Drive/TF_zscore_F_u.txt',sep='\t',header=0)
fff=pd.read_csv('drive/My Drive/gene_zscore_F_u.txt',sep='\t',header=0)
mm=pd.read_csv('drive/My Drive/TF_zscore_M_u.txt',sep='\t',header=0)
mmm=pd.read_csv('drive/My Drive/gene_zscore_M_u.txt',sep='\t',header=0)

sex_tf_diffB=ff.merge(mm,on='tf')
sex_tf_diffB.columns=['TF','F_case','F_cont','Fdiff','M_case','M_cont','Mdiff']
# sex_tf_diffB.to_csv('drive/My Drive/TF_sex_diff.txt',sep='\t',index=0)


sex_gene_diffB=fff.merge(mmm,on='gene')
sex_gene_diffB.columns=['gene','F_case','F_cont','Fdiff','M_case','M_cont','Mdiff']
# sex_gene_diffB.to_csv('drive/My Drive/gene_sex_diff.txt',sep='\t',index=0)

In [ ]:
ff=pd.read_csv('drive/My Drive/TF_zscore_F_large_u.txt',sep='\t',header=0)
fff=pd.read_csv('drive/My Drive/gene_zscore_F_large_u.txt',sep='\t',header=0)
mm=pd.read_csv('drive/My Drive/TF_zscore_M_large_u.txt',sep='\t',header=0)
mmm=pd.read_csv('drive/My Drive/gene_zscore_M_large_u.txt',sep='\t',header=0)

sex_tf_diffA=ff.merge(mm,on='tf')
sex_tf_diffA.columns=['TF','F_case','F_cont','Fdiff','M_case','M_cont','Mdiff']
# sex_tf_diffA.to_csv('drive/My Drive/TF_sex_diff.txt',sep='\t',index=0)


sex_gene_diffA=fff.merge(mmm,on='gene')
sex_gene_diffA.columns=['gene','F_case','F_cont','Fdiff','M_case','M_cont','Mdiff']
# sex_gene_diffA.to_csv('drive/My Drive/gene_sex_diff.txt',sep='\t',index=0)

In [ ]:
steve=sex_gene_diffB.merge(sex_gene_diffA,on='gene')
c=steve[(steve['Mdiff_x']>.4)&(steve['Mdiff_y']>.4)].gene.dropna().astype(object)
# c
genes_str='\n'.join(c)
ENRICHR_URL = 'http://maayanlab.cloud/Enrichr/addList'
query_string = '?userListId=%s&backgroundType=%s'

description = 'Example gene list'
payload = {
    'list': (None, genes_str),
    'description': (None, description)
}

response = requests.post(ENRICHR_URL, files=payload)
if not response.ok:
    raise Exception('Error analyzing gene list')

data = json.loads(response.text)

user_list_id = data['userListId']
gene_set_library = 'GO_Biological_Process_2018'
ENRICHR_URL = 'http://maayanlab.cloud/Enrichr/enrich'
response = requests.get(
    ENRICHR_URL + query_string % (user_list_id, gene_set_library)
  )
if not response.ok:
    raise Exception('Error fetching enrichment results')

data = json.loads(response.text);
processBio=[]
pvals=[]
for i in range(1,10):
    processBio.append(data['GO_Biological_Process_2018'][i][1])
    pvals.append(data['GO_Biological_Process_2018'][i][2])
plt.figure()
y_pos = np.arange(len(processBio))
plt.barh(y_pos, -np.log10(pvals))
plt.yticks(ticks=y_pos,labels=processBio);
plt.gca().invert_yaxis()  
plt.xlabel('-log(p-value)')

In [ ]:
steve=sex_gene_diffB.merge(sex_gene_diffA,on='gene')
c=steve[(steve['Fdiff_x']>.4)&(steve['Fdiff_y']>.4)].gene.dropna().astype(object)
# c
genes_str='\n'.join(c)
ENRICHR_URL = 'http://maayanlab.cloud/Enrichr/addList'
query_string = '?userListId=%s&backgroundType=%s'

description = 'Example gene list'
payload = {
    'list': (None, genes_str),
    'description': (None, description)
}

response = requests.post(ENRICHR_URL, files=payload)
if not response.ok:
    raise Exception('Error analyzing gene list')

data = json.loads(response.text)

user_list_id = data['userListId']
gene_set_library = 'GO_Biological_Process_2018'
ENRICHR_URL = 'http://maayanlab.cloud/Enrichr/enrich'
response = requests.get(
    ENRICHR_URL + query_string % (user_list_id, gene_set_library)
  )
if not response.ok:
    raise Exception('Error fetching enrichment results')

data = json.loads(response.text);
processBio=[]
pvals=[]
for i in range(1,10):
    processBio.append(data['GO_Biological_Process_2018'][i][1])
    pvals.append(data['GO_Biological_Process_2018'][i][2])
plt.figure()
y_pos = np.arange(len(processBio))
plt.barh(y_pos, -np.log10(pvals))
plt.yticks(ticks=y_pos,labels=processBio);
plt.gca().invert_yaxis()  
plt.xlabel('-log(p-value)')

In [ ]:
sex_tf_diffA=sex_tf_diffA.sort_values('TF')
sex_tf_diffB=sex_tf_diffB.sort_values('TF')
sex_gene_diffA=sex_gene_diffA.sort_values('gene')
sex_gene_diffB=sex_gene_diffB.sort_values('gene')

In [ ]:
sex_tf_diffA

In [ ]:
plt.figure(figsize=(8, 8))
# p1=plt.bar(sex_tf_diffB.index,sex_tf_diffB['Mdiff'])
# p2=plt.bar(sex_tf_diffA.index,sex_tf_diffA['Mdiff'],bottom=sex_tf_diffB['Mdiff']) 
# p3=plt.bar(sex_tf_diffB.index,sex_tf_diffB['Fdiff'])
# p4=plt.bar(sex_tf_diffA.index,sex_tf_diffA['Fdiff'],bottom=sex_tf_diffB['Fdiff'])
# plt.legend((p1[0], p2[0],p3[0],p4[0]), ('smallMen', 'largeMen','smallWomen','largeWomen'))

p1=plt.scatter(sex_tf_diffB['Mdiff'],-sex_tf_diffA['Mdiff'],alpha=.25) ## minus because one is case-control, other control-case!!
p2=plt.scatter(sex_tf_diffB['Fdiff'],-sex_tf_diffA['Fdiff'],alpha=.25)
# p3=plt.scatter(sex_tf_diffA['Mdiff'],sex_tf_diffA['Fdiff'],alpha=.25) ## minus because one is case-control, other control-case!!
# p4=plt.scatter(sex_tf_diffB['Mdiff'],sex_tf_diffB['Fdiff'],alpha=.25)
plt.ylabel('LTRC case-control TF targeting diff')
plt.xlabel('LTCOPD case-control TF targeting diff')
plt.legend((p1, p2),('Male','Female'))#,p3,p4),('Male','Female','LTRC','LTCOPD'))
plt.savefig("fig.png",dpi=300,bbox_inches = "tight")


In [ ]:
# from scipy.stats import spearmanr
# from scipy.stats import zscore

In [ ]:
rhoM, pvalM = np.corrcoef(sex_tf_diffB['Mdiff'], -sex_tf_diffA['Mdiff'])
# pvalM
rhoF, pvalF = np.corrcoef(sex_tf_diffB['Fdiff'], -sex_tf_diffA['Fdiff'])
# pvalF
print([rhoM[1],rhoF[1]])

rhoM, pvalM = stats.spearmanr(sex_tf_diffB['Mdiff'], -sex_tf_diffA['Mdiff'])
# pvalM
rhoF, pvalF = stats.spearmanr(sex_tf_diffB['Fdiff'], -sex_tf_diffA['Fdiff'])
# pvalF
print([rhoM,rhoF])

# rhoM, pvalM = stats.spearmanr(sex_tf_diffA['Mdiff'], sex_tf_diffA['Fdiff'])
# # pvalM
# rhoF, pvalF = stats.spearmanr(sex_tf_diffA['Mdiff'], sex_tf_diffA['Fdiff'])
# # pvalF
# print([rhoM,rhoF])

# rhoM, pvalM = stats.spearmanr(sex_tf_diffB['Mdiff'], sex_tf_diffB['Fdiff'])
# # pvalM
# rhoF, pvalF = stats.spearmanr(sex_tf_diffB['Mdiff'], sex_tf_diffB['Fdiff'])
# # pval
# print([rhoM,rhoF])

In [ ]:
jeff=pd.DataFrame([sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[0:50],
sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[586:636],
sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[0:50],
sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[586:636],
sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[0:50],
sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[586:636],
sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[0:50],
sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[586:636]])

jeff.T

In [ ]:
pp=jeff.iloc[2].tolist()

print(*pp, sep='\n')

In [ ]:
jeff=[set(sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[0:50]),
set(sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[586:636]),
set(sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[0:50]),
set(sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[586:636]),
set(sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[0:50]),
set(sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[586:636]),
set(sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[0:50]),
set(sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[586:636])]

https://github.com/gecko984/supervenn:
> The numbers at the bottom show the sizes (cardinalities) of all intersections, which we will call chunks. The sizes of sets and their intersections (chunks) are up to proportion, but the order of elements is not preserved


In [ ]:
!pip install supervenn
from supervenn import supervenn

In [ ]:
plt.figure(figsize=(20, 10))
species_names=['LTRC_Fdiff_up','LTRC_Fdiff_down','LTRC_Mdiff_up','LTRC_Mdiff_down',
      'LTCOPD_Fdiff_up','LTCOPD_Fdiff_down','LTCOPD_Mdiff_up','LTCOPD_Mdiff_down']

cc=supervenn(jeff, species_names, rotate_col_annotations=True)#,
          # col_annotations_area_height=1.2, sets_ordering='minimize gaps',
          # min_width_for_annotation=180)

In [ ]:
#A=LTCOPD, cont-case
pp=set(sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[0:50]).difference(set(sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[0:50]))
print(*pp, sep='\n')
print('\n')
pp=set(sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[586:636]).difference(set(sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[586:636]))
print(*pp, sep='\n')

In [ ]:
pp=set(sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[0:50]).difference(set(sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[0:50]))
print(*pp, sep='\n')
print('\n')
pp=set(sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[586:636]).difference(set(sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[586:636]))
print(*pp, sep='\n')

In [ ]:
pp=set(sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[0:50]).difference(set(sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[0:50]))
print(*pp, sep='\n')
print('\n')
pp=set(sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[586:636]).difference(set(sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[586:636]))
print(*pp, sep='\n')

In [ ]:
pp=set(sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[0:50]).difference(set(sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[0:50]))
print(*pp, sep='\n')
print('\n')
pp=set(sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[586:636]).difference(set(sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[586:636]))
print(*pp, sep='\n')

In [ ]:
plt.figure(figsize=(8, 8))

p1=plt.scatter(bb['Mdiff_x'],bb['Mdiff_y'],alpha=.5)
p2=plt.scatter(bb['Fdiff_x'],bb['Fdiff_y'],alpha=.25)
plt.ylabel('LTRC control-case gene targeting diff')
plt.xlabel('LTCOPD control-case gene targeting diff')
plt.legend((p1, p2),('Male','Female'))

In [ ]:
rhoM, pvalM = np.corrcoef(bb['Mdiff_x'], bb['Mdiff_y'])
# pvalM
rhoF, pvalF = np.corrcoef(bb['Fdiff_x'], bb['Fdiff_y'])
# pvalF
print([rhoM[1],rhoF[1]])

rhoM, pvalM = stats.spearmanr(bb['Mdiff_x'], bb['Mdiff_y'])
# pvalM
rhoF, pvalF = stats.spearmanr(bb['Fdiff_x'], bb['Fdiff_y'])
# pvalF
print([rhoM,rhoF])

In [ ]:
jeff=pd.DataFrame([sex_gene_diffA.sort_values(by='Fdiff')['gene'].values[0:100],
sex_gene_diffA.sort_values(by='Fdiff')['gene'].values[(len(sex_gene_diffA.Fdiff)-100):len(sex_gene_diffA.Fdiff)],
sex_gene_diffA.sort_values(by='Mdiff')['gene'].values[0:100],
sex_gene_diffA.sort_values(by='Mdiff')['gene'].values[(len(sex_gene_diffA.Mdiff)-100):len(sex_gene_diffA.Mdiff)],
sex_gene_diffB.sort_values(by='Fdiff')['gene'].values[0:100],
sex_gene_diffB.sort_values(by='Fdiff')['gene'].values[(len(sex_gene_diffB.Fdiff)-100):len(sex_gene_diffB.Fdiff)],
sex_gene_diffB.sort_values(by='Mdiff')['gene'].values[0:100],
sex_gene_diffB.sort_values(by='Mdiff')['gene'].values[(len(sex_gene_diffB.Mdiff)-100):len(sex_gene_diffB.Mdiff)]])

# jeff=jeff.T


In [ ]:

pp=jeff[1].tolist()

print(*pp, sep='\n')

In [ ]:
jeff2=[set(sex_gene_diffA.sort_values(by='Fdiff')['gene'].values[0:100]),
set(sex_gene_diffA.sort_values(by='Fdiff')['gene'].values[(len(sex_gene_diffA.Fdiff)-100):len(sex_gene_diffA.Fdiff)]),
set(sex_gene_diffA.sort_values(by='Mdiff')['gene'].values[0:100]),
set(sex_gene_diffA.sort_values(by='Mdiff')['gene'].values[(len(sex_gene_diffA.Mdiff)-100):len(sex_gene_diffA.Mdiff)]),
set(sex_gene_diffB.sort_values(by='Fdiff')['gene'].values[0:100]),
set(sex_gene_diffB.sort_values(by='Fdiff')['gene'].values[(len(sex_gene_diffB.Fdiff)-100):len(sex_gene_diffB.Fdiff)]),
set(sex_gene_diffB.sort_values(by='Mdiff')['gene'].values[0:100]),
set(sex_gene_diffB.sort_values(by='Mdiff')['gene'].values[(len(sex_gene_diffB.Mdiff)-100):len(sex_gene_diffB.Mdiff)])]

In [ ]:
plt.figure(figsize=(20, 10))
species_names=['large_Fdiff_up','large_Fdiff_down','large_Mdiff_up','large_Mdiff_down',
      'Fdiff_up','Fdiff_down','Mdiff_up','Mdiff_down']

supervenn(jeff2, species_names, rotate_col_annotations=True)#,
          # col_annotations_area_height=1.2, sets_ordering='minimize gaps',
          # min_width_for_annotation=180)

##MALE V FEMALE CASE

In [ ]:
MC=pd.read_csv('drive/My Drive/LARGE_sex_panda/MC_LTRC.txt',names=['TF','gene','motif','force'],sep=' ')
FC=pd.read_csv('drive/My Drive/LARGE_sex_panda/FC_LTRC.txt',names=['TF','gene','motif','force'],sep='\t')


In [ ]:
TFsum_3=MC.groupby(['TF']).agg({'force':sum})
TFsum_1=FC.groupby(['TF']).agg({'force':sum})
TFsum_1['force']=zscore(TFsum_1['force'])
TFsum_3['force']=zscore(TFsum_3['force']) ##reduces range from 1500 to 3

geneSum_3=MC.groupby(['gene']).agg({'force':sum})
geneSum_1=FC.groupby(['gene']).agg({'force':sum})
geneSum_3['force']=zscore(geneSum_3['force'])
geneSum_1['force']=zscore(geneSum_1['force'])

mmm=TFsum_3.merge(TFsum_1, left_on=['TF'], right_on=['TF'])
mmmm=geneSum_3.merge(geneSum_1, left_on=['gene'], right_on=['gene'])

# del [[MC, MNC]]
# gc.collect()
MC=pd.DataFrame()
MNC=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()


mmm['diff']=mmm.force_x-mmm.force_y
mmm = mmm.sort_values(by ='diff' )


mmmm['diff']=mmmm.force_x-mmmm.force_y
mmmm = mmmm.sort_values(by ='diff' )
mmm.to_csv('drive/My Drive/LARGE_sex_panda/TF_zscore_Case.txt',sep='\t')
mmmm.to_csv('drive/My Drive/LARGE_sex_panda/gene_zscore_Case.txt',sep='\t',index=0)

##MALE V FEMALE CONTROL


In [ ]:
MNC=pd.read_csv('data/LTRC/sex/MNC_LTRC.txt',names=['TF','gene','motif','force'],sep=' ')
FNC=pd.read_csv('data/LTRC/sex/FNC_LTRC.txt',names=['TF','gene','motif','force'],sep=' ')

In [ ]:
TFsum_4=MNC.groupby(['TF']).agg({'force':sum})
TFsum_2=FNC.groupby(['TF']).agg({'force':sum})
TFsum_2['force']=zscore(TFsum_2['force'])
TFsum_4['force']=zscore(TFsum_4['force']) ##reduces range from 2500 to 4

geneSum_4=FNC.groupby(['gene']).agg({'force':sum})
geneSum_2=MNC.groupby(['gene']).agg({'force':sum})
geneSum_4['force']=zscore(geneSum_4['force'])
geneSum_2['force']=zscore(geneSum_2['force'])

mmm=TFsum_4.merge(TFsum_2, left_on=['TF'], right_on=['TF'])
mmmm=geneSum_4.merge(geneSum_2, left_on=['gene'], right_on=['gene'])

# del [[FC, FNC]]
# gc.collect()
FC=pd.DataFrame()
FNC=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()


mmm['diff']=mmm.force_x-mmm.force_y
mmm = mmm.sort_values(by ='diff' )


mmmm['diff']=mmmm.force_x-mmmm.force_y
mmmm = mmmm.sort_values(by ='diff' )
mmm.to_csv('drive/My Drive/LARGE_sex_panda/TF_zscore_Control.txt',sep='\t')
mmmm.to_csv('drive/My Drive/LARGE_sex_panda/gene_zscore_Control.txt',sep='\t',index=0)

In [ ]:
ff=pd.read_csv('data/LTRC/sex/TF_zscore_Case.txt',sep='\t',header=0)
fff=pd.read_csv('data/LTRC/sex/gene_zscore_Case.txt',sep='\t',header=0)
mm=pd.read_csv('data/LTRC/sex/TF_zscore_Control.txt',sep='\t',header=0)
mmm=pd.read_csv('data/LTRC/sex/gene_zscore_Control.txt',sep='\t',header=0)
sex_tf_diff=ff.merge(mm,on='TF')

a=plt.hist(sex_tf_diff['diff_x'],bins=100,label='Case')
b=plt.hist(sex_tf_diff['diff_y'],bins=100,label='Control')
plt.title('LTRC Sex-specific Male-Female TF degree diff')
plt.legend(loc="best")


#negative control
create 4 randomly chosen subsets to test difference of differences between panda nets

use other colab for more memory:
https://colab.research.google.com/drive/1ebLeRX6Gwywgs1Yn_gcL8TYufgkfbchX#scrollTo=wS_rwFkacl74






In [ ]:
# !pip install GEOparse
# import GEOparse
# gse = GEOparse.get_GEO(geo="GSE76925", destdir="./")
# illumina_labels=pd.read_csv('drive/My Drive/Colab Notebooks dcm/GPL10558_illumina_prb_loc.txt',sep='\t',index_col=0,header=None,names=['loc','symbol'])

# gpl = gse.gpls[next(iter(gse.gpls))]
# data = gse.pivot_samples(values="VALUE")#, gpl=gpl, annotation_column="Symbol")
# data.sort_index

LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/ppi/ppi_complete.txt',sep='\t',header=None)
LCL_ppi.to_csv('drive/My Drive/ppi_complete.txt',sep='\t',index=False,header=False)

coding_LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/motifs/Hugo_motifCellLine.txt',sep='\t',header=None)
coding_LCL_motif.to_csv('drive/My Drive/coding_LCL_motif.txt',sep='\t',index=False,header=False)

motif_data='drive/My Drive/Colab Notebooks/coding_LCL_motif.txt'
ppi_data='drive/My Drive/Colab Notebooks/ppi_complete.txt'

# plt.hist(nullA.values.flatten(),alpha=.5,log=True)
# plt.hist(nullB.values.flatten(),alpha=.5,log=True)
# plt.hist(nullC.values.flatten(),alpha=.5,log=True)
# plt.hist(nullD.values.flatten(),alpha=.5,log=True)
# plt.xlabel('test')


In [ ]:
null=data.merge(illumina_labels,left_index=True,right_index=True)
null.index=null['symbol']
del null['loc'], null['symbol']

In [ ]:
def negCont(nullA,nullB,i):
  panda_nullA = Panda(nullA, motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
  # np.save('drive/My Drive/Panda_nullA.npy',panda_nullA.export_panda_results)
  panda_nullB = Panda(nullB, motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
  # np.save('drive/My Drive/Panda_nullB.npy',panda_nullB.export_panda_results)

  subset_panda_results3 = panda_nullA.export_panda_results.sort_values(by=['force'], ascending=False)
  subset_panda_results1 = panda_nullB.export_panda_results.sort_values(by=['force'], ascending=False)
  del subset_panda_results3['motif'],subset_panda_results1['motif']
  del panda_nullA, panda_nullB

  ##aggregate TF link sum and zscore
  TFsum_3=subset_panda_results3.groupby(['tf']).agg({'force':sum})
  TFsum_1=subset_panda_results1.groupby(['tf']).agg({'force':sum})
  TFsum_1['force']=zscore(TFsum_1['force'])
  TFsum_3['force']=zscore(TFsum_3['force'])

  ##aggregate gene link sum and zscore
  geneSum_3=subset_panda_results3.groupby(['gene']).agg({'force':sum})
  geneSum_1=subset_panda_results1.groupby(['gene']).agg({'force':sum})
  geneSum_3['force']=zscore(geneSum_3['force'])
  geneSum_1['force']=zscore(geneSum_1['force'])

  ##merge
  nnn=TFsum_3.merge(TFsum_1, left_on=['tf'], right_on=['tf'])
  nnnn=geneSum_3.merge(geneSum_1, left_on=['gene'], right_on=['gene'])

  del [[subset_panda_results3, subset_panda_results1]]
  gc.collect()
  subset_panda_results3=pd.DataFrame()
  subset_panda_results1=pd.DataFrame()

  ##diff
  nnn['diff']=nnn.force_x-nnn.force_y
  # nnn = nnn.sort_values(by ='diff' )
  del nnn['force_x'], nnn['force_y']
  nnn=nnn.sort_index(axis=0)
  nnn=nnn.T
  nnnn['diff']=nnnn.force_x-nnnn.force_y
  # nnnn = nnnn.sort_values(by ='diff' )
  del nnnn['force_x'], nnnn['force_y']
  nnnn=nnnn.sort_index(axis=0)
  nnnn=nnnn.T
  if i==1:
    nnn.to_csv('drive/My Drive/Colab Notebooks/model_output/TF_zscore_rand_CLE.txt',sep='\t',header=True,index=None)
    nnnn.to_csv('drive/My Drive/Colab Notebooks/model_output/gene_zscore_rand_CLE.txt',sep='\t',header=True,index=None)
  elif i!=1:
    nnn.to_csv('drive/My Drive/Colab Notebooks/model_output/TF_zscore_rand_CLE.txt',sep='\t',header=None,index=None,mode='a')
    nnnn.to_csv('drive/My Drive/Colab Notebooks/model_output/gene_zscore_rand_CLE.txt',sep='\t',header=None,index=None,mode='a')
  
  del nnn, nnnn
  gc.collect()



In [ ]:
for i in range(1,10):
  ## four random group exact same size as LTCOPD cases and controls
  null=data.merge(illumina_labels,left_index=True,right_index=True)
  null.index=null['symbol']
  del null['loc'], null['symbol']
  j=np.random.choice(151, size=(67, ),replace=False) ##67 total males
  nullA=null.iloc[:,j.tolist()] ##reduce to random 68
  null2=null.drop(nullA.columns,axis=1) ## create second group without these 68

  k=np.random.choice(j,15,replace=False) ##15 male controls
  nullB=null.iloc[:,k.tolist()] ##take 15 of 68 for male control random
  nullA=nullA.drop(nullB.columns,axis=1) ## remove these 15 from male cases

  l=np.random.choice(null2.shape[1], size=(84, ),replace=False) ##84 total females
  nullC=null2.iloc[:,l.tolist()] ##subset leftover from first null to 85
  m=np.random.choice(l[l<85],25,replace=False) ##25 controls
  nullD=nullC.iloc[:,m.tolist()] ##take 25 for female random control
  nullC=nullC.drop(nullD.columns,axis=1)

  negCont(nullA,nullB,i)
  negCont(nullC,nullD,i+1)

  print('saved iteration='+str(i))

In [ ]:
TF_diffNull=pd.read_csv('drive/My Drive/Colab Notebooks/model_output/TF_zscore_rand_CLE.txt',sep='\t')
gene_diffNull=pd.read_csv('drive/My Drive/Colab Notebooks/model_output/gene_zscore_rand_CLE.txt',sep='\t')
TF_diffNull=TF_diffNull.T
gene_diffNull=gene_diffNull.T

In [ ]:
for i, col in enumerate(TF_diffNull.columns):
    # plt.figure(i)
    tmp=sns.kdeplot(TF_diffNull[col])

In [ ]:
for i, col in enumerate(gene_diffNull.columns):
    # plt.figure(i)
    tmp=sns.kdeplot(gene_diffNull[col])

# LTRC negCont

In [ ]:
LTRC_exp=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/age_sex_LTRC_XXX.txt',sep='\t',index_col=0)
LTRC_exp_key=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/age_sex_LTRC_XXXkey.txt',sep='\t')
data=np.divide(LTRC_exp,LTRC_exp_key)
data=data.groupby(data.index).mean()

motif_data='drive/My Drive/coding_LCL_motif.txt'
ppi_data='drive/My Drive/ppi_complete.txt'

LTCOPD_genes=pd.read_csv('drive/My Drive/Colab Notebooks/LTCOPD_genes.txt',index_col=None,header=None,sep='\t')
data1=np.log2(data.dropna(how='all'))
data1=data1.merge(LTCOPD_genes,left_index=True,right_on=0)
data1.index=data1[0]
data1.replace([np.inf, -np.inf], np.nan,inplace=True)
data1.replace(np.nan, 0,inplace=True)
del data1[0]

In [ ]:
FC=data1.iloc[:,data1.columns.str.contains('_0_1')]
FNC=data1.iloc[:,data1.columns.str.contains('_0_0')]
MC=data1.iloc[:,data1.columns.str.contains('_1_1')]
MNC=data1.iloc[:,data1.columns.str.contains('_1_0')]

In [ ]:
def negCont(nullA,nullB,i,timestr,motif_data,ppi_data):
  panda_nullA = Panda(nullA, motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=True, keep_expression_matrix = False,modeProcess = 'intersection')
  # np.save('drive/My Drive/Panda_nullA.npy',panda_nullA.export_panda_results)
  gc.collect()
  panda_nullB = Panda(nullB, motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = True, remove_missing=True, keep_expression_matrix = False,modeProcess = 'intersection')
  # np.save('drive/My Drive/Panda_nullB.npy',panda_nullB.export_panda_results)
  uniq_tfs=np.intersect1d(panda_nullA.ppi_tfs,panda_nullA.motif_tfs)
  uniq_genes=np.intersect1d(panda_nullA.motif_genes,panda_nullA.expression_genes)
  tfs = np.tile(uniq_tfs, (len(uniq_genes), 1)).flatten()
  genes = np.repeat(uniq_genes,panda_nullA.num_tfs)
  forceA = panda_nullA.motif_matrix.flatten(order='F')
  forceB = panda_nullB.motif_matrix.flatten(order='F')
  export_panda_results = pd.DataFrame({'tf':tfs, 'gene': genes,'forceA': forceA,'forceB': forceB})#.sort_values(by=['force'], ascending=False)

  # subset_panda_resultsA = panda_nullA.export_panda_results.sort_values(by=['force'], ascending=False)
  # subset_panda_resultsB = panda_nullB.export_panda_results.sort_values(by=['force'], ascending=False)
  # del subset_panda_resultsA['motif'],subset_panda_resultsB['motif']
  del panda_nullA, panda_nullB, forceA, forceB

  ##aggregate TF link sum and zscore
  TFsum_A=export_panda_results.groupby(['tf']).agg({'forceA':sum})
  TFsum_B=export_panda_results.groupby(['tf']).agg({'forceB':sum})
  TFsum_B['force']=zscore(TFsum_B['forceB'])
  TFsum_A['force']=zscore(TFsum_A['forceA'])

  ##aggregate gene link sum and zscore
  geneSum_A=export_panda_results.groupby(['gene']).agg({'forceA':sum})
  geneSum_B=export_panda_results.groupby(['gene']).agg({'forceB':sum})
  geneSum_A['force']=zscore(geneSum_A['forceA'])
  geneSum_B['force']=zscore(geneSum_B['forceB'])

  ##merge
  sumTFdiff=TFsum_A.merge(TFsum_B, left_on=['tf'], right_on=['tf'])
  sumGenediff=geneSum_A.merge(geneSum_B, left_on=['gene'], right_on=['gene'])

  del export_panda_results, tfs, genes,forceA,forceB #[[subset_panda_resultsA, subset_panda_resultsB]]
  gc.collect()
  export_panda_results=pd.DataFrame()
  # subset_panda_resultsA=pd.DataFrame()
  # subset_panda_resultsB=pd.DataFrame()

  ##diff
  sumTFdiff['diff']=sumTFdiff.force_x-sumTFdiff.force_y
  # sumTFdiff = sumTFdiff.sort_values(by ='diff' )
  del sumTFdiff['force_x'], sumTFdiff['force_y']
  sumTFdiff=sumTFdiff.sort_index(axis=0)
  sumTFdiff=sumTFdiff.T
  sumGenediff['diff']=sumGenediff.force_x-sumGenediff.force_y
  # sumGenediff = sumGenediff.sort_values(by ='diff' )
  del sumGenediff['force_x'], sumGenediff['force_y']
  sumGenediff=sumGenediff.sort_index(axis=0)
  sumGenediff=sumGenediff.T
  if i==1:
    sumTFdiff.to_csv('drive/My Drive/Colab Notebooks/model_output/null/TF_zscore_rand_'+timestr+'.txt',sep='\t',header=True,index=None)
    sumGenediff.to_csv('drive/My Drive/Colab Notebooks/model_output/null/gene_zscore_rand_'+timestr+'.txt',sep='\t',header=True,index=None)
  elif i!=1:
    sumTFdiff.to_csv('drive/My Drive/Colab Notebooks/model_output/null/TF_zscore_rand_'+timestr+'.txt',sep='\t',header=None,index=None,mode='a')
    sumGenediff.to_csv('drive/My Drive/Colab Notebooks/model_output/null/gene_zscore_rand_'+timestr+'.txt',sep='\t',header=None,index=None,mode='a')
  del sumTFdiff, sumGenediff
  gc.collect()

In [ ]:
inits=100
for i in range(1,inits):
    ## form two random groups of exact size as initial cases and controls
    # data=pd.read_csv(expression_file,sep='\t',index_col=0)
    j=np.random.choice(data1.shape[1], size=(MC.shape[1]+MNC.shape[1], ),replace=False) ## total males
    nullA=data1.iloc[:,j.tolist()] ##reduce to random set
    # null2=data.drop(nullA.columns,axis=1) ## create second group without these

    k=np.random.choice(j,MNC.shape[1],replace=False) ## male controls
    nullB=data1.iloc[:,k.tolist()] ##remove controls from male pop
    nullA=nullA.drop(nullB.columns,axis=1) ## remove these from male cases

    # l=np.random.choice(null2.shape[1], size=(FC.shape[1]+FNC.shape[1], ),replace=False) ##84 total females
    # nullC=null2.iloc[:,l.tolist()] ##subset leftover from first null to 85
    # m=np.random.choice(l[l<FC.shape[1]+1+FNC.shape[1]],FNC.shape[1],replace=False) ##25 controls
    # nullD=nullC.iloc[:,m.tolist()] ##take 25 for female random control
    # nullC=nullC.drop(nullD.columns,axis=1)

    sumTFdiff,sumGenediff=negCont(nullA,nullB,i,timestr,motif_data,ppi_data)
    # negCont(nullC,nullD,i+1)

    print('saved iteration='+str(i))

In [ ]:

# f_pval =[]
# m_pval = []#pd.DataFrame()
# f_pval = np.array(f_pval, dtype = np.float32)
# m_pval = np.array(m_pval, dtype = np.float32)

# for i, col in enumerate(TF_diffNull.columns):
#   t0f, p0f=stats.ttest_ind(mmm['diff'],TF_diffNull[col],nan_policy='omit',equal_var=False)
#   t0m, p0m=stats.ttest_ind(nnn['diff'],TF_diffNull[col],nan_policy='omit',equal_var=False)
#   f_pval=np.append(f_pval,p0f)
#   m_pval=np.append(m_pval,p0m)
# plt.hist([m_pval])

In [ ]:
TF_diffNull=pd.read_csv('drive/My Drive/Colab Notebooks/model_output/TF_zscore_rand_1000.txt',sep='\t',header=0)
gene_diffNull=pd.read_csv('drive/My Drive/Colab Notebooks/model_output/gene_zscore_rand_1000.txt',sep='\t',header=0)
TF_diffNull=TF_diffNull.T
gene_diffNull=gene_diffNull.T
gene_diffNull.shape

# TF_diffNull=pd.read_csv('drive/My Drive/TF_zscore_rand_camb.txt',sep='\t',header=0)
# gene_diffNull=pd.read_csv('drive/My Drive/gene_zscore_rand_camb.txt',sep='\t',header=0)
# TF_diffNull=TF_diffNull.T
# gene_diffNull=gene_diffNull.T
# gene_diffNull.shape

In [ ]:
male=pd.DataFrame()
for i in range(0,200,2):
  xx=pd.DataFrame([TF_diffNull[TF_diffNull[i]>.5].index,TF_diffNull[TF_diffNull[i]<-.5].index])
  if not xx.empty:
    print(xx)
    male=np.append(male,xx)

In [ ]:
male

In [ ]:
for i in range(1,201,2):
  xx=pd.DataFrame([TF_diffNull[TF_diffNull[i]>.5][0],TF_diffNull[TF_diffNull[i]<-.5][0]])
  if not xx.empty:
    print(i)

In [ ]:
TF_diffNull

In [ ]:
[sex_tf_diff[(sex_tf_diff['diff_y'])>.4].tf.values,sex_tf_diff[(sex_tf_diff['diff_y'])<-0.4].tf.values] #,sex_tf_diff[np.abs(sex_tf_diff['diff_y'])>.4].tf]

In [ ]:
for i, col in enumerate(TF_diffNull.columns):
    # plt.figure(i)
    tmp=sns.distplot(TF_diffNull[col])

In [ ]:
for i, col in enumerate(gene_diffNull.columns):
    # plt.figure(i)
    tmp=sns.distplot(gene_diffNull[col])

In [ ]:
hist=np.histogram(np.array(TF_diffNull).flatten())#,log=False)
stats.rv_histogram(hist)

In [ ]:
plt.hist(TF_diffNull.unstack(),log=True)

In [ ]:
plt.hist(TF_diffNull.unstack())

In [ ]:
sns.distplot(np.median(TF_diffNull,axis=1))
sns.distplot(np.mean(TF_diffNull,axis=1))

In [ ]:
hist=np.histogram(np.array(TF_diffNull).flatten(),100)#,log=False)
TF_dNull=stats.rv_histogram(hist)
sampled_list = random.sample(set(mmm['diff']), 100)
stat2, pvalue2 = stats.kstest(sampled_list, cdf=TF_dNull.cdf)#,args=(weight1, mean1, stdv1, mean2, stdv2))
pvalue2

In [ ]:
TFpvalue=pd.DataFrame()
for i, col in enumerate(TF_diffNull.columns):
  hist=np.histogram(np.array(TF_diffNull[col]))#.flatten())#,log=False)
  TF_dNull=stats.rv_histogram(hist)
  sampled_list = random.sample(set(mmm['diff']), 100)
  stat2, pvalue2 = stats.kstest(sampled_list, cdf=TF_dNull.cdf)#,args=(weight1, mean1, stdv1, mean2, stdv2))
  TFpvalue[i+1]=[pvalue2] #np.concatenate([pvalue,pvalue2],axis=0)
TFpvalue=TFpvalue.T
print(TFpvalue)

In [ ]:
TFpvalue[TFpvalue<.005].dropna()

In [ ]:
Gpvalue=pd.DataFrame()
for i, col in enumerate(gene_diffNull.columns):
  hist=np.histogram(np.array(gene_diffNull[col]))#.flatten())#,log=False)
  gene_dNull=stats.rv_histogram(hist)
  sampled_list = random.sample(set(mmmm['diff']), 100)
  stat2, pvalue2 = stats.kstest(sampled_list, cdf=gene_dNull.cdf)#,args=(weight1, mean1, stdv1, mean2, stdv2))
  Gpvalue[i+1]=[pvalue2] #np.concatenate([pvalue,pvalue2],axis=0)
Gpvalue=Gpvalue.T
print(Gpvalue)

In [ ]:
Gpvalue[Gpvalue<.005].dropna()

#Age case v control (65yo)

In [ ]:
# del data1['loc']
# Age=(gse.phenotype_data['characteristics_ch1.0.age'].values).astype('float')>64
# len(Age.astype('str'))
# data1=pd.read_csv('drive/My Drive/Colab Notebooks/GSE76925.txt',sep='\t',index_col=0)
# data1
# data1.columns=[Age.astype('str')+gse.phenotype_data['characteristics_ch1.5.copd'].values]

# data1.loc[:,'Truecase'].to_csv('OldCase.txt',sep='\t',index=True,header=False)
# data1.loc[:,'Truecont'].to_csv('OldCont.txt',sep='\t',index=True,header=False)
[data1.loc[:,'Truecase'].shape,#.to_csv('OldCase.txt',sep='\t',index=True,header=False)
data1.loc[:,'Truecont'].shape,

# # data4=data1[data1['age']<=65]
# # del  data4['loc']
# # data4.columns=[gse.phenotype_data['characteristics_ch1.1.Sex']+gse.phenotype_data['characteristics_ch1.5.copd']]
# data1.loc[:,'Falsecase'].to_csv('YoungCase.txt',sep='\t',index=True,header=False)
# data1.loc[:,'Falsecont'].to_csv('YoungCont.txt',sep='\t',index=True,header=False)
data1.loc[:,'Falsecase'].shape,#.to_csv('YoungCase.txt',sep='\t',index=True,header=False)
data1.loc[:,'Falsecont'].shape]

In [ ]:
LCL_ppi=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/ppi/ppi_complete.txt',sep='\t',header=None)
LCL_ppi.to_csv('ppi_complete.txt',sep='\t',index=False,header=False)

coding_LCL_motif=pd.read_csv('https://granddb.s3.amazonaws.com/optPANDA/motifs/Hugo_motifCellLine.txt',sep='\t',header=None)
coding_LCL_motif.to_csv('coding_LCL_motif.txt',sep='\t',index=False,header=False)

motif_data='coding_LCL_motif.txt'
ppi_data='ppi_complete.txt'

In [ ]:
panda_obj1 = Panda('OldCase.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
np.save('drive/My Drive/Panda_OldCase.npy',panda_obj1.export_panda_results)
del panda_obj1
panda_obj2 = Panda('OldCont.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
np.save('drive/My Drive/Panda_OldCont.npy',panda_obj2.export_panda_results)
del panda_obj2
panda_obj3 = Panda('YoungCase.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
np.save('drive/My Drive/Panda_YoungCase.npy',panda_obj3.export_panda_results)
del panda_obj3
panda_obj4 = Panda('YoungCont.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
np.save('drive/My Drive/Panda_YoungCont.npy',panda_obj4.export_panda_results)
del panda_obj4


In [ ]:
Panda_OldCase=np.load('drive/My Drive/Colab Notebooks/Panda_OldCase.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_OldCont=np.load('drive/My Drive/Colab Notebooks/Panda_OldCont.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_OldCont=pd.DataFrame(Panda_OldCont)
Panda_OldCont.columns=['tf','gene','motif','force']
Panda_OldCase=pd.DataFrame(Panda_OldCase)
Panda_OldCase.columns=['tf','gene','motif','force']

Panda_YoungCase=np.load('drive/My Drive/Colab Notebooks/Panda_YoungCase.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_YoungCont=np.load('drive/My Drive/Colab Notebooks/Panda_YoungCont.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
Panda_YoungCont=pd.DataFrame(Panda_YoungCont)
Panda_YoungCont.columns=['tf','gene','motif','force']
Panda_YoungCase=pd.DataFrame(Panda_YoungCase)
Panda_YoungCase.columns=['tf','gene','motif','force']

#TF and gene specific zscore diff OLD, YOUNG

In [ ]:
from scipy.stats import zscore
# Panda_nullA.panda_results = pd.DataFrame(Panda_nullA.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results3 = Panda_OldCase.sort_values(by=['force'], ascending=False)
# subset_panda_results3['merge']=subset_panda_results3.tf+'-'+subset_panda_results3.gene

# Panda_nullB.panda_results = pd.DataFrame(Panda_nullB.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results1 = Panda_OldCont.sort_values(by=['force'], ascending=False)
# subset_panda_results1['merge']=subset_panda_results1.tf+'-'+subset_panda_results1.gene
del subset_panda_results3['motif'],subset_panda_results1['motif']
TFsum_3=subset_panda_results3.groupby(['tf']).agg({'force':sum})
TFsum_1=subset_panda_results1.groupby(['tf']).agg({'force':sum})
TFsum_1['force']=zscore(TFsum_1['force'])
TFsum_3['force']=zscore(TFsum_3['force']) ##reduces range from 1500 to 3

geneSum_3=subset_panda_results3.groupby(['gene']).agg({'force':sum})
geneSum_1=subset_panda_results1.groupby(['gene']).agg({'force':sum})
geneSum_3['force']=zscore(geneSum_3['force'])
geneSum_1['force']=zscore(geneSum_1['force'])

mmm=TFsum_3.merge(TFsum_1, left_on=['tf'], right_on=['tf'])
mmmm=geneSum_3.merge(geneSum_1, left_on=['gene'], right_on=['gene'])

del [[subset_panda_results3, subset_panda_results1,Panda_OldCont,Panda_OldCase]]
gc.collect()
subset_panda_results3=pd.DataFrame()
subset_panda_results1=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()


mmm['diff']=mmm.force_x-mmm.force_y
mmm = mmm.sort_values(by ='diff' )


mmmm['diff']=mmmm.force_x-mmmm.force_y
mmmm = mmmm.sort_values(by ='diff' )
mmm.to_csv('drive/My Drive/TF_zscore_O.txt',sep='\t')
mmmm.to_csv('drive/My Drive/gene_zscore_O.txt',sep='\t',index=0)

In [ ]:

# Panda_nullA.panda_results = pd.DataFrame(Panda_nullA.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results3 = Panda_YoungCase.sort_values(by=['force'], ascending=False)
# subset_panda_results3['merge']=subset_panda_results3.tf+'-'+subset_panda_results3.gene

# Panda_nullB.panda_results = pd.DataFrame(Panda_nullB.export_panda_results, columns=['tf','gene','motif','force'])
subset_panda_results1 = Panda_YoungCont.sort_values(by=['force'], ascending=False)
# subset_panda_results1['merge']=subset_panda_results1.tf+'-'+subset_panda_results1.gene
del subset_panda_results3['motif'],subset_panda_results1['motif']
TFsum_3=subset_panda_results3.groupby(['tf']).agg({'force':sum})
TFsum_1=subset_panda_results1.groupby(['tf']).agg({'force':sum})
TFsum_1['force']=zscore(TFsum_1['force'])
TFsum_3['force']=zscore(TFsum_3['force'])

geneSum_3=subset_panda_results3.groupby(['gene']).agg({'force':sum})
geneSum_1=subset_panda_results1.groupby(['gene']).agg({'force':sum})
geneSum_3['force']=zscore(geneSum_3['force'])
geneSum_1['force']=zscore(geneSum_1['force'])

nnn=TFsum_3.merge(TFsum_1, left_on=['tf'], right_on=['tf'])
nnnn=geneSum_3.merge(geneSum_1, left_on=['gene'], right_on=['gene'])

del [[subset_panda_results3, subset_panda_results1,Panda_YoungCase,Panda_YoungCont]]
gc.collect()
subset_panda_results3=pd.DataFrame()
subset_panda_results1=pd.DataFrame()
Panda_Mcont=pd.DataFrame()
Panda_Mcase=pd.DataFrame()


nnn['diff']=nnn.force_x-nnn.force_y
nnn = nnn.sort_values(by ='diff' )


nnnn['diff']=nnnn.force_x-nnnn.force_y
nnnn = nnnn.sort_values(by ='diff' )
nnn.to_csv('drive/My Drive/TF_zscore_Y.txt',sep='\t')
nnnn.to_csv('drive/My Drive/gene_zscore_Y.txt',sep='\t')


In [ ]:
ff=pd.read_csv('drive/My Drive/TF_zscore_O.txt',sep='\t',header=0)
fff=pd.read_csv('drive/My Drive/gene_zscore_O.txt',sep='\t',header=0)
mm=pd.read_csv('drive/My Drive/TF_zscore_Y.txt',sep='\t',header=0)
mmm=pd.read_csv('drive/My Drive/gene_zscore_Y.txt',sep='\t',header=0)
sex_tf_diff=ff.merge(mm,on='tf')
# sex_gene_diff=fff.merge(mmm,on='gene')
 
plt.figure(figsize=(15,6))
sex_tf_diff['sexdiff']=sex_tf_diff['diff_x']-sex_tf_diff['diff_y']
# sex_tf_diff=sex_tf_diff.sort_values(by=['sexdiff'])
sex_tf_diff=sex_tf_diff.sort_values(by=['tf'])
sns.barplot(data=sex_tf_diff,y='sexdiff',x='tf')

In [ ]:
a=plt.hist(sex_tf_diff['diff_x'],bins=100,label='<65',color='c')
b=plt.hist(sex_tf_diff['diff_y'],bins=100,label='>=65',color='m')
plt.title('Age-specific Ca-Co TF degree diff')
plt.legend(loc="best")

In [ ]:
# [sex_tf_diff[np.abs(sex_tf_diff['diff_x'])>.4].tf,
sex_tf_diff#[np.abs(sex_tf_diff['diff_y'])>.4].tf]

#LTCOPD combat

In [ ]:
LTCOPD_exp=pd.read_csv('drive/MyDrive/harvard/LARGE_COPD/LTCOPD_combat.txt',sep='\t',index_col=0)
# LTRC_exp=pd.read_csv('drive/MyDrive/harvard/LARGE_COPD/LTRC_qsmooth_combat.txt',sep='\t',index_col=0)
LTCOPD_exp=LTCOPD_exp.reset_index().dropna().set_index('index')
dd=pd.read_csv('drive/MyDrive/harvard/LARGE_COPD/LTCOPD_genes.txt',sep='\t',index_col=0)
LTCOPD_exp.index=dd['x']

In [ ]:
# LTCOPD_genes=pd.read_csv('drive/My Drive/Colab Notebooks/LTCOPD_genes.txt',index_col=None,header=None,sep='\t')
LTCOPD_exp=(LTCOPD_exp.dropna(how='all'))
data2=LTCOPD_exp.merge(pd.DataFrame(data1.index.tolist()),left_index=True,right_on=0)
data2.index=data2[0]
data2.replace([np.inf, -np.inf], np.nan,inplace=True)
data2.replace(np.nan, 0,inplace=True)
data2=data2.groupby(data2.index).mean()

In [ ]:

FC=data2.iloc[:,data2.columns.str.contains('_F_case')]
FNC=data2.iloc[:,data2.columns.str.contains('_F_cont')]
MC=data2.iloc[:,data2.columns.str.contains('_M_case')]
MNC=data2.iloc[:,data2.columns.str.contains('_M_cont')]
print([FC.shape,FNC.shape,MC.shape,MNC.shape])

FC.to_csv('Fcase.txt',sep='\t',index=True,header=False)
FNC.to_csv('Fcont.txt',sep='\t',index=True,header=False)
MC.to_csv('Mcase.txt',sep='\t',index=True,header=False)
MNC.to_csv('Mcont.txt',sep='\t',index=True,header=False)

In [ ]:
panda_obj1 = Panda('Fcase.txt', F_motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/FC_LTCOPD.npy',np.round(panda_obj1.export_panda_results,decimals=3))
del panda_obj1
panda_obj2 = Panda('Mcase.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/MC_LTCOPD.npy',np.round(panda_obj2.export_panda_results,decimals=3))
del panda_obj2
panda_obj3 = Panda('Fcont.txt', F_motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/FNC_LTCOPD.npy',np.round(panda_obj3.export_panda_results,decimals=3))
del panda_obj3
panda_obj4 = Panda('Mcont.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = False,modeProcess = 'intersection')
np.save('drive/My Drive/Colab Notebooks/model_output/MNC_LTCOPD.npy',np.round(panda_obj4.export_panda_results,decimals=3))
del panda_obj4
# panda_obj5 = Panda('FcaseFULL.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
# np.save('drive/My Drive/Panda_FcaseFULL.npy',panda_obj5.export_panda_results)
# del panda_obj5
# panda_obj6 = Panda('FcontFULL.txt', motif_data, ppi_data,computing='gpu',precision='single',save_tmp=False,save_memory = False, remove_missing=False, keep_expression_matrix = True,modeProcess = 'intersection')
# np.save('drive/My Drive/Panda_FcontFULL.npy',panda_obj6.export_panda_results)
# del panda_obj6


In [ ]:
MC=np.load('drive/My Drive/Colab Notebooks/model_output/MC_LTCOPD.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
MNC=np.load('drive/My Drive/Colab Notebooks/model_output/MNC_LTCOPD.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
MNC=pd.DataFrame(MNC)
MNC.columns=['tf','gene','motif','force']
MC=pd.DataFrame(MC)
MC.columns=['tf','gene','motif','force']

TFsum_3=MC.groupby(['tf']).agg({'force':sum})
TFsum_1=MNC.groupby(['tf']).agg({'force':sum})
TFsum_1['force']=zscore(TFsum_1['force'])
TFsum_3['force']=zscore(TFsum_3['force']) ##reduces range from 1500 to 3

geneSum_3=MC.groupby(['gene']).agg({'force':sum})
geneSum_1=MNC.groupby(['gene']).agg({'force':sum})
geneSum_3['force']=zscore(geneSum_3['force'])
geneSum_1['force']=zscore(geneSum_1['force'])

mmm=TFsum_3.merge(TFsum_1, left_on=['tf'], right_on=['tf'])
mmmm=geneSum_3.merge(geneSum_1, left_on=['gene'], right_on=['gene'])

# del [[MC, MNC]]
# gc.collect()
MC=pd.DataFrame()
MNC=pd.DataFrame()
# Panda_FcontFULL=pd.DataFrame()
# Panda_FcontFULL=pd.DataFrame()


mmm['diff']=mmm.force_x-mmm.force_y
mmm = mmm.sort_values(by ='diff' )


mmmm['diff']=mmmm.force_x-mmmm.force_y
mmmm = mmmm.sort_values(by ='diff' )
mmm.to_csv('drive/My Drive/TF_zscore_M.txt',sep='\t')
mmmm.to_csv('drive/My Drive/gene_zscore_M.txt',sep='\t')

In [ ]:
FC=np.load('drive/My Drive/Colab Notebooks/model_output/FC_LTCOPD.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
FNC=np.load('drive/My Drive/Colab Notebooks/model_output/FNC_LTCOPD.npy',allow_pickle=True)#,sep='\t',index_col=0,header=None,names=['loc','symbol'])
FNC=pd.DataFrame(FNC)
FNC.columns=['tf','gene','motif','force']
FC=pd.DataFrame(FC)
FC.columns=['tf','gene','motif','force']

TFsum_4=FC.groupby(['tf']).agg({'force':sum})
TFsum_2=FNC.groupby(['tf']).agg({'force':sum})
TFsum_2['force']=zscore(TFsum_2['force'])
TFsum_4['force']=zscore(TFsum_4['force']) ##reduces range from 2500 to 4

geneSum_4=FC.groupby(['gene']).agg({'force':sum})
geneSum_2=FNC.groupby(['gene']).agg({'force':sum})
geneSum_4['force']=zscore(geneSum_4['force'])
geneSum_2['force']=zscore(geneSum_2['force'])

nnn=TFsum_4.merge(TFsum_2, left_on=['tf'], right_on=['tf'])
nnnn=geneSum_4.merge(geneSum_2, left_on=['gene'], right_on=['gene'])

# del [[FC, FNC]]
# gc.collect()
FC=pd.DataFrame()
FNC=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()
Panda_FcontFULL=pd.DataFrame()


nnn['diff']=nnn.force_x-nnn.force_y
nnn = nnn.sort_values(by ='diff' )


nnnn['diff']=mmmm.force_x-nnnn.force_y
nnnn = nnnn.sort_values(by ='diff' )
nnn.to_csv('drive/My Drive/TF_zscore_F.txt',sep='\t')
nnnn.to_csv('drive/My Drive/gene_zscore_F.txt',sep='\t')

###load and compare Clue-Reg results

In [ ]:
LTRC_male=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/LTRC_Monly_clueReg.csv',header=0)
LTRC_female=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/LTRC_Fonly_clueReg.csv',header=0)
LTCOPD_male=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/LTCOPD_Monly_clueReg.csv',header=0)
LTCOPD_female=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/LTCOPD_Fonly_clueReg.csv',header=0)

female=LTRC_female[['Drug','Overlap','Cosine','P-value']].merge(LTCOPD_female[['Drug','Overlap','Cosine','P-value']],right_on='Drug',left_on='Drug')
male=LTRC_male[['Drug','Overlap','Cosine','P-value']].merge(LTCOPD_male[['Drug','Overlap','Cosine','P-value']],right_on='Drug',left_on='Drug')

LTRC_rx=LTRC_female[['Drug','Overlap','Cosine','P-value']].merge(LTRC_male[['Drug','Overlap','Cosine','P-value']],right_on='Drug',left_on='Drug')
LTCOPD_rx=LTCOPD_female[['Drug','Overlap','Cosine','P-value']].merge(LTCOPD_male[['Drug','Overlap','Cosine','P-value']],right_on='Drug',left_on='Drug')

plt.figure(figsize=(8,8))

plt.scatter(male.Cosine_x,male.Cosine_y,label='male'),
plt.scatter(female.Cosine_x,female.Cosine_y,label='female')

# plt.scatter(LTRC_rx.Cosine_x,LTRC_rx.Cosine_y,label='LTRC'),
# plt.scatter(LTCOPD_rx.Cosine_x,LTCOPD_rx.Cosine_y,label='LTCOPD')

plt.legend(loc='best')
# plt.ylim(-.4,0)
# plt.xlim(-.4,0)
# plt.plot([-1, 0], [-1, 0], 'k--')
plt.xlabel('LTRC cosine')
plt.ylabel('LTCOPD cosine')

rhoM, pvalM = np.corrcoef(female.Cosine_x,female.Cosine_y)
# pvalM
rhoF, pvalF = np.corrcoef(male.Cosine_x,male.Cosine_y)
# pvalF
print([rhoM[1],rhoF[1]])

rhoM, pvalM = stats.spearmanr(female.Cosine_x,female.Cosine_y)
# pvalM
rhoF, pvalF = stats.spearmanr(male.Cosine_x,male.Cosine_y)
# pvalF
print([rhoM,rhoF])

# rhoM, pvalM = stats.spearmanr(LTRC_rx.Cosine_x,LTRC_rx.Cosine_y)
# # pvalM
# rhoF, pvalF = stats.spearmanr(LTCOPD_rx.Cosine_x,LTCOPD_rx.Cosine_y)
# # pvalF
# print([rhoM,rhoF])

# rhoM, pvalM = stats.spearmanr(LTRC_rx.Cosine_x,LTRC_rx.Cosine_y)
# # pvalM
# rhoF, pvalF = stats.spearmanr(LTCOPD_rx.Cosine_x,LTCOPD_rx.Cosine_y)
# # pval
# print([rhoM,rhoF])

In [ ]:
!pip install chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default='notebook'


trace_men = go.Scatter(x=male['Cosine_x'],
                  y=male['Cosine_y'],text=male['Drug'],
                  name='male',mode='markers',
                  marker=dict(color='#A2D5F2'))

trace_women = go.Scatter(x=female['Cosine_x'],
                y=female['Cosine_y'],text=female['Drug'],
                name='female',mode='markers',
                marker=dict(color='#ffcdd2'))

trace_ltrc = go.Scatter(x=LTRC_rx['Cosine_x'],
                y=LTRC_rx['Cosine_y'],text=LTRC_rx['Drug'],
                name='LTRC_rx',mode='markers')#,
                # marker=dict(color='#ffcdd2'))

trace_ltcopd = go.Scatter(x=LTCOPD_rx['Cosine_x'],
                y=LTCOPD_rx['Cosine_y'],text=LTCOPD_rx['Drug'],
                name='LTCOPD_rx',mode='markers')#,
                # marker=dict(color='#ffcdd2'))

data = [trace_men, trace_women,trace_ltrc,trace_ltcopd]

layout = go.Layout(title="cosine",
                xaxis=dict(title='cosine'),
                yaxis=dict(title='cosine'))

fig = go.Figure(data=data, layout=layout,layout_yaxis_range=[-.6,0],layout_xaxis_range=[-.6,0])
fig.add_trace(go.Scatter(
    x=[-1,0,],
    y=[-1,0],name='',
))
fig.update_layout(
    autosize=False,
    width=500,
    height=500)
fig.show(renderer="colab")

In [ ]:
LTRC_male=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/LTRC_Monly_clueReg.csv',header=0)
LTRC_female=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/LTRC_Fonly_clueReg.csv',header=0)
LTCOPD_male=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/LTCOPD_Monly_clueReg.csv',header=0)
LTCOPD_female=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/LTCOPD_Fonly_clueReg.csv',header=0)

female=LTRC_female[['Drug','Overlap','Cosine','P-value']].merge(LTCOPD_female[['Drug','Overlap','Cosine','P-value']],right_on='Drug',left_on='Drug')
male=LTRC_male[['Drug','Overlap','Cosine','P-value']].merge(LTCOPD_male[['Drug','Overlap','Cosine','P-value']],right_on='Drug',left_on='Drug')

LTRC_rx=LTRC_female[['Drug','Overlap','Cosine','P-value']].merge(LTRC_male[['Drug','Overlap','Cosine','P-value']],right_on='Drug',left_on='Drug')
LTCOPD_rx=LTCOPD_female[['Drug','Overlap','Cosine','P-value']].merge(LTCOPD_male[['Drug','Overlap','Cosine','P-value']],right_on='Drug',left_on='Drug')

plt.figure(figsize=(8,8))

plt.scatter(male.Cosine_x,male.Cosine_y,label='male'),
plt.scatter(female.Cosine_x,female.Cosine_y,label='female')

plt.scatter(LTRC_rx.Cosine_x,LTRC_rx.Cosine_y,label='LTRC'),
plt.scatter(LTCOPD_rx.Cosine_x,LTCOPD_rx.Cosine_y,label='LTCOPD')

plt.legend(loc='best')
# plt.ylim(-.4,0)
# plt.xlim(-.4,0)
# plt.plot([-1, 0], [-1, 0], 'k--')
plt.xlabel('LTRC cosine')
plt.ylabel('LTCOPD cosine')

rhoM, pvalM = np.corrcoef(female.Cosine_x,female.Cosine_y)
# pvalM
rhoF, pvalF = np.corrcoef(male.Cosine_x,male.Cosine_y)
# pvalF
print([rhoM[1],rhoF[1]])

rhoM, pvalM = stats.spearmanr(female.Cosine_x,female.Cosine_y)
# pvalM
rhoF, pvalF = stats.spearmanr(male.Cosine_x,male.Cosine_y)
# pvalF
print([rhoM,rhoF])

rhoM, pvalM = stats.spearmanr(LTRC_rx.Cosine_x,LTRC_rx.Cosine_y)
# pvalM
rhoF, pvalF = stats.spearmanr(LTCOPD_rx.Cosine_x,LTCOPD_rx.Cosine_y)
# pvalF
print([rhoM,rhoF])

rhoM, pvalM = stats.spearmanr(LTRC_rx.Cosine_x,LTRC_rx.Cosine_y)
# pvalM
rhoF, pvalF = stats.spearmanr(LTCOPD_rx.Cosine_x,LTCOPD_rx.Cosine_y)
# pval
print([rhoM,rhoF])

In [ ]:
!pip install chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default='notebook'


trace_men = go.Scatter(x=male['Cosine_x'],
                  y=male['Cosine_y'],text=male['Drug'],
                  name='male',mode='markers',
                  marker=dict(color='#A2D5F2'))

trace_women = go.Scatter(x=female['Cosine_x'],
                y=female['Cosine_y'],text=female['Drug'],
                name='female',mode='markers',
                marker=dict(color='#ffcdd2'))

trace_ltrc = go.Scatter(x=LTRC_rx['Cosine_x'],
                y=LTRC_rx['Cosine_y'],text=LTRC_rx['Drug'],
                name='LTRC_rx',mode='markers')#,
                # marker=dict(color='#ffcdd2'))

trace_ltcopd = go.Scatter(x=LTCOPD_rx['Cosine_x'],
                y=LTCOPD_rx['Cosine_y'],text=LTCOPD_rx['Drug'],
                name='LTCOPD_rx',mode='markers')#,
                # marker=dict(color='#ffcdd2'))

data = [trace_men, trace_women,trace_ltrc,trace_ltcopd]

layout = go.Layout(title="cosine",
                xaxis=dict(title='cosine'),
                yaxis=dict(title='cosine'))

fig = go.Figure(data=data, layout=layout,layout_yaxis_range=[-.4,0],layout_xaxis_range=[-.4,0])
fig.add_trace(go.Scatter(
    x=[-1,0,],
    y=[-1,0],name='',
))
fig.update_layout(
    autosize=False,
    width=500,
    height=500)
fig.show(renderer="colab")

In [ ]:

LTRC_male=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/ltrc_M_copd_io.txt',header=0,sep='\t')
LTRC_female=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/ltrc_F_copd_io.txt',header=0,sep='\t')
LTCOPD_male=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/ltcopd_M_copd_io.txt',header=0,sep='\t')
LTCOPD_female=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/ltcopd_F_copd_io.txt',header=0,sep='\t')

female=LTRC_female.merge(LTCOPD_female,right_on='Name',left_on='Name')
male=LTRC_male.merge(LTCOPD_male,right_on='Name',left_on='Name')

LTRC_rx=LTRC_female.merge(LTRC_male,right_on='Name',left_on='Name')
LTCOPD_rx=LTCOPD_female.merge(LTCOPD_male,right_on='Name',left_on='Name')



In [ ]:
jeff11=[set(LTRC_female.Name),
set(LTRC_male.Name),
set(LTCOPD_female.Name),
set(LTCOPD_male.Name)]

plt.figure(figsize=(20, 10))
species_names=['LTRC_F','LTRC_M','LTCOPD_F','LTCOPD_M']

supervenn(jeff11, species_names, rotate_col_annotations=True)#,
          # col_annotations_area_height=1.2, sets_ordering='minimize gaps',
          # min_width_for_annotation=180)

In [ ]:
jeff33=[set(LTRC_female.Name),
set(LTRC_male.Name),
set(LTCOPD_female.Name),
set(LTCOPD_male.Name),
set(sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[0:50]),
set(sex_tf_diffA.sort_values(by='Fdiff')['TF'].values[586:636]),
set(sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[0:50]),
set(sex_tf_diffA.sort_values(by='Mdiff')['TF'].values[586:636]),
set(sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[0:50]),
set(sex_tf_diffB.sort_values(by='Fdiff')['TF'].values[586:636]),
set(sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[0:50]),
set(sex_tf_diffB.sort_values(by='Mdiff')['TF'].values[586:636])]

In [ ]:
plt.figure(figsize=(20, 10))
species_names=['LTRC_F_io','LTRC_M_io','LTCOPD_F_io','LTCOPD_M_io',
               'LTCOPD_F_up','LTCOPD_F_down','LTCOPD_M_up','LTCOPD_M_down',
               'LTRC_F_down','LTRC_F_up','LTCOPD_M_down','LTRC_M_up',]

supervenn(jeff33, species_names, rotate_col_annotations=True)#,
          # col_annotations_area_height=1.2, sets_ordering='minimize gaps',
          # min_width_for_annotation=180)

#Diff Exp

In [ ]:
ltrc_F_copd=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/F_LTRC_de_061821.txt',sep='\t')#,names=['ensemble','AveExpr','logFC','P.Value','adj.P.Val'],skiprows=1)
ltrc_M_copd=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/M_LTRC_de_061821.txt',sep='\t')#,names=['ensemble','AveExpr','logFC','P.Value','adj.P.Val'],skiprows=1)
ltrc_genes=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/LTRC_genes_chr.txt',sep='\t',names=['ensemble','ID','chr'],skiprows=1)
ltrc_F_copd.index=(ltrc_F_copd.index).str.split('.').str[0].tolist()
ltrc_M_copd.index=(ltrc_M_copd.index).str.split('.').str[0].tolist()
ltrc_F_copd=ltrc_F_copd.merge(ltrc_genes,left_index=True,right_on='ensemble')
ltrc_M_copd=ltrc_M_copd.merge(ltrc_genes,left_index=True,right_on='ensemble')
# ltrc_F_copd['ID']=ltrc_F_copd.index
# ltrc_M_copd['ID']=ltrc_M_copd.index


ltcopd_F_copd=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/F_LTCOPD_de_062221.txt',sep='\t')
ltcopd_M_copd=pd.read_csv('drive/My Drive/harvard/LARGE_COPD/M_LTCOPD_de_062221.txt',sep='\t')#,nrows=1500

In [ ]:
from natsort import index_natsorted
plt.figure(figsize=(20,5))
# print(len(ltrc_M_copd.chr[ltrc_M_copd['adj.P.Val']<0.01]))
# ltrc_M_copd['chrm']=ltrc_M_copd.chr.str.split('chr').str[1].tolist()
cc=(ltrc_M_copd[ltrc_M_copd['adj.P.Val']<0.01])
cc=cc.sort_values(by="chr",key=lambda x: np.argsort(index_natsorted(cc["chr"])))
# cc.sort(lambda x,y: x-y, key=lambda x: Set_Chr_Nr_(x))#_values()
print(len(cc))
pp=len(np.unique(cc.chr))
zz=plt.hist(cc.chr,bins=pp)

In [ ]:
# from natsort import index_natsorted
plt.figure(figsize=(20,5))
# print(len(ltrc_M_copd.chr[ltrc_M_copd['adj.P.Val']<0.01]))
# ltrc_M_copd['chrm']=ltrc_M_copd.chr.str.split('chr').str[1].tolist()
cc=(ltrc_F_copd[ltrc_F_copd['adj.P.Val']<0.01])
cc=cc.sort_values(by="chr",key=lambda x: np.argsort(index_natsorted(cc["chr"])))
# cc.sort(lambda x,y: x-y, key=lambda x: Set_Chr_Nr_(x))#_values()
print(len(cc))
pp=len(np.unique(cc.chr))
zz=plt.hist(cc.chr,bins=pp)

In [ ]:
cc

In [ ]:
from natsort import index_natsorted
plt.figure(figsize=(20,5))
# print(len(ltrc_M_copd.chr[ltrc_M_copd['adj.P.Val']<0.01]))
# ltrc_M_copd['chrm']=ltrc_M_copd.chr.str.split('chr').str[1].tolist()
cc=(ltcopd_F_copd[ltcopd_F_copd['adj.P.Val']<0.05])
# cc=cc.sort_values(by="chr",key=lambda x: np.argsort(index_natsorted(cc["chr"])))
# cc.sort(lambda x,y: x-y, key=lambda x: Set_Chr_Nr_(x))#_values()
print(len(cc))
# pp=len(np.unique(cc.chr))
zz=plt.hist(cc['adj.P.Val'],bins=50)

In [ ]:
from natsort import index_natsorted
plt.figure(figsize=(20,5))
# print(len(ltrc_M_copd.chr[ltrc_M_copd['adj.P.Val']<0.01]))
# ltrc_M_copd['chrm']=ltrc_M_copd.chr.str.split('chr').str[1].tolist()
cc=(ltcopd_M_copd[ltcopd_M_copd['adj.P.Val']<0.05])
# cc=cc.sort_values(by="chr",key=lambda x: np.argsort(index_natsorted(cc["chr"])))
# cc.sort(lambda x,y: x-y, key=lambda x: Set_Chr_Nr_(x))#_values()
print(len(cc))
# pp=len(np.unique(cc.chr))
zz=plt.hist(cc['adj.P.Val'],bins=50)

In [ ]:
female_DE=ltrc_F_copd.merge(ltcopd_F_copd,on='ID')
male_DE=ltrc_M_copd.merge(ltcopd_M_copd,on='ID')
female_DE=female_DE.dropna()
male_DE=male_DE.dropna()
[len(male_DE),len(female_DE)]

In [ ]:
!pip install chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go

import plotly.io as pio
pio.renderers.default='notebook'
import plotly

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(male_DE['AveExpr_x'],male_DE['AveExpr_y'],label='male'),
plt.scatter(female_DE['AveExpr_x'],female_DE['AveExpr_y'],label='female')
plt.legend(loc='best')
plt.ylim(0,14)
plt.xlim(0,14)
# plt.ylim(-.4,0)
# plt.xlim(-.4,0)
plt.plot([0, 14], [0, 14], 'k--')
plt.xlabel('LTRC Avg_Expr')
plt.ylabel('LTCOPD Avg_Expr')

In [ ]:
trace_men = go.Scatter(x=male_DE['AveExpr_x'],
                  y=male_DE['AveExpr_y'],text=male_DE['ID'],
                  name='male',mode='markers',
                  marker=dict(color='#A2D5F2'))

trace_women = go.Scatter(x=female_DE['AveExpr_x'],
                y=female_DE['AveExpr_y'],text=female_DE['ID'],
                name='female',mode='markers',
                marker=dict(color='#ffcdd2'))

data = [trace_men, trace_women]

layout = go.Layout(title="avg_exp",
                xaxis=dict(title='LTRC avg_exp'),
                yaxis=dict(title='LTCOPD avg_exp'))

fig = go.Figure(data=data, layout=layout,layout_yaxis_range=[0,14],layout_xaxis_range=[0,14])
fig.add_trace(go.Scatter(
    x=[0, 14],
    y=[0, 14],name='',
))
fig.update_layout(
    autosize=False,
    width=500,
    height=500)
fig.show(renderer="colab")

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(male_DE['P.Value_x'],male_DE['P.Value_y'],label='male'),
plt.scatter(female_DE['P.Value_x'],female_DE['P.Value_y'],label='female')
plt.legend(loc='best')

plt.plot([0, 0.02], [0, .02], 'k--')
plt.ylim(0,.025)
plt.xlim(0,.0015)
plt.xlabel('LTRC P.val')
plt.ylabel('LTCOPD P.val')

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(male_DE['adj.P.Val_x'],male_DE['adj.P.Val_y'],label='male'),
plt.scatter(female_DE['adj.P.Val_x'],female_DE['adj.P.Val_y'],label='female')
plt.legend(loc='best')
# plt.ylim(0,14)
# plt.xlim(0,14)
plt.plot([0, .5], [0, .5], 'k--')
plt.xlabel('LTRC adj.p.val')
plt.ylabel('LTCOPD adj.p.val')

In [ ]:
plt.figure(figsize=(5,5))
plt.scatter(male_DE['t_x'],male_DE['t_y'],label='male'),
plt.scatter(female_DE['t_x'],female_DE['t_y'],label='female')
plt.legend(loc='best')
# plt.ylim(-.4,0)
# plt.xlim(-.4,0)
plt.plot([-6, 6], [-6, 6], 'k--')
plt.xlabel('LTRC direction of effect (t)')
plt.ylabel('LTCOPD direction of effect (t)')

In [ ]:
trace_men = go.Scatter(x=male_DE['t_x'],
                  y=-male_DE['t_y'],text=male_DE['ID'],
                  name='male',mode='markers',
                  marker=dict(color='#A2D5F2',size=np.abs(np.log((male_DE['adj.P.Val_x']*male_DE['adj.P.Val_y'])))))

trace_women = go.Scatter(x=female_DE['t_x'],
                y=-female_DE['t_y'],text=female_DE['ID'],
                name='female',mode='markers',
                marker=dict(color='#ffcdd2',size=np.abs(np.log(female_DE['adj.P.Val_x']*female_DE['adj.P.Val_y']))))

data = [trace_men, trace_women]

layout = go.Layout(title="direction & magnitude of effect ",
                xaxis=dict(title='LTRC t'),
                yaxis=dict(title='LTCOPD t'))

fig = go.Figure(data=data, layout=layout,layout_yaxis_range=[-6,6],layout_xaxis_range=[-6,6])
fig.add_trace(go.Scatter(
    x=[-6, 6],
    y=[-6, 6],name='',
))
fig.update_layout(#title={ 'xanchor': 'center'},#'text': 'direction & magnitude of effect '},#\n (size=abs(log(male p-val * female p-val)'},
    autosize=False,
    width=500,
    height=500)
fig.show(renderer="colab")

In [ ]:
LTRC_men = go.Scatter(x=male_DE['B_x'],
                  y=male_DE['P.Value_x'],text=male_DE['ID'],
                  name='LTRC male',mode='markers',
                  marker=dict(color='#A2D5F2'),yaxis='y2')#,size=np.abs(np.log(male_DE['adj.P.Val_x']))),yaxis='y2')

LTRC_women = go.Scatter(x=female_DE['B_x'],
                y=female_DE['P.Value_x'],text=female_DE['ID'],
                name='LTRC female',mode='markers',
                marker=dict(color='#ffcdd2'),yaxis='y2')#,size=np.abs(np.log(female_DE['adj.P.Val_x']))))


LTCOPD_men = go.Scatter(x=male_DE['B_y'],
                  y=male_DE['P.Value_y'],text=male_DE['ID'],
                  name='LTCOPD male',mode='markers',
                  marker=dict(color='#A2D5F2',symbol='square-dot'))#,size=(male_DE['adj.P.Val_y'])))

LTCOPD_women = go.Scatter(x=female_DE['B_y'],
                y=female_DE['P.Value_y'],text=female_DE['ID'],
                name='LTCOPD female',mode='markers',
                marker=dict(color='#ffcdd2',symbol='square-dot'))#,size=(female_DE['adj.P.Val_y']))))



data = [LTRC_men, LTRC_women,LTCOPD_men,LTCOPD_women]

layout = go.Layout(title="avg_exp",
                xaxis=dict(title='direcion of effect (t)'),
                yaxis=dict(title='p-val'),
                yaxis2=dict(title='log p-val',type='log',overlaying='y',side='right'))

fig = go.Figure(data=data, layout=layout)#,layout_yaxis_range=[-6,6],layout_xaxis_range=[-6,6])
# fig.add_trace(go.Scatter(
#     x=[-6, 6],
#     y=[-6, 6],name='',
# ))
fig.update_layout(
    autosize=False,
    width=500,
    height=500)
fig.show(renderer="colab")

# plotly.sign_in('dcolinmorgan', 'nAZ6TvqQqkrkAN7lpV6v')
# plot_url = plotly.plotly.plot(fig)

In [ ]:
trace_men = go.Scatter(x=male_DE['P.Value_x'],
                  y=male_DE['P.Value_y'],text=male_DE['ID'],
                  name='male',mode='markers',
                  marker=dict(color='#A2D5F2',symbol=((male_DE['t_x']*male_DE['t_y'])>1)))

trace_women = go.Scatter(x=female_DE['P.Value_x'],
                y=female_DE['P.Value_y'],text=female_DE['ID'],
                name='female',mode='markers',
                marker=dict(color='#ffcdd2',symbol=((female_DE['t_x']*female_DE['t_y'])>1)))

data = [trace_men, trace_women]

layout = go.Layout(title="direction & magnitude of effect ",
                xaxis=dict(title='LTRC p-val'),
                yaxis=dict(title='LTCOPD p-val'))

fig = go.Figure(data=data, layout=layout)#,layout_yaxis_range=[-6,6],layout_xaxis_range=[-6,6])
# fig.add_trace(go.Scatter(
#     x=[-6, 6],
#     y=[-6, 6],name='',
# ))
fig.update_layout(#title={ 'xanchor': 'center'},#'text': 'direction & magnitude of effect '},#\n (size=abs(log(male p-val * female p-val)'},
    autosize=False,
    width=500,
    height=500)
fig.show(renderer="colab")

In [ ]:
jeff3=[set(ltcopd_F_copd['ID']),
set(ltcopd_M_copd['ID']),
set(ltrc_F_copd['ID']),
set(ltrc_M_copd['ID'])]

In [ ]:
plt.figure(figsize=(20, 10))
species_names=['ltcopd_Fdiff','ltcopd_Mdiff','ltrc_Fdiff','ltrc_Mdiff']

supervenn(jeff3, species_names, rotate_col_annotations=True)#,
          # col_annotations_area_height=1.2, sets_ordering='minimize gaps',
          # min_width_for_annotation=180)

In [ ]:
jeff4=pd.DataFrame([ltcopd_F_copd.sort_values('P.Value')['ID'].values[0:100],
                    ltcopd_M_copd.sort_values('P.Value')['ID'].values[0:100],
                    ltrc_F_copd.sort_values('P.Value')['ID'].values[0:100],
                    ltrc_M_copd.sort_values('P.Value')['ID'].values[0:100]])

jeff4.T

In [ ]:

pp=jeff4.iloc[3].dropna().tolist()

print(*pp, sep='\n')

In [ ]:
jeff22=[set(LTRC_female.Name),
set(LTRC_male.Name),
set(LTCOPD_female.Name),
set(LTCOPD_male.Name),

set(ltcopd_F_copd['ID'][1:100]),
set(ltcopd_M_copd['ID'][1:100]),
set(ltrc_F_copd['ID'][1:100]),
set(ltrc_M_copd['ID'][1:100])]

In [ ]:
plt.figure(figsize=(20, 10))
species_names=['reg_LTRC_F','reg_LTRC_M','reg_LTCOPD_F','reg_LTCOPD_M','io_ltcopd_F','io_ltcopd_F','io_ltrc_large_M','io_ltrc_M',]
supervenn(jeff22, species_names, rotate_col_annotations=True)#,
          # col_annotations_area_height=1.2, sets_ordering='minimize gaps',
          # min_width_for_annotation=180)

In [ ]:
j=-1
jeff=pd.DataFrame()
# titleArray=['Binary network 100 genes','Continous network 1 100 genes','Continous network 2 100 genes','Continous network 3 100 genes']
# traces= glob.glob('drive/My Drive/harvard/LARGE_COPD/*_de_061821.txt')
# traces=np.sort(traces)
for j in range(0,4):


    # for net in #[KRCCNetBin1,KRCCNetCont1,KRCCNetCont2,KRCCNetCont3]:
    # j=j+1
    # if '.txt' in(file):
        # net=pd.read_csv(file,sep='\t',usecols=[3,8,9])
    # else:
    #     net=pd.read_csv(file,sep=',',usecols=[3,8,9])
    # net=net.sort_values('adj.P.Val').dropna()[0:500]
    # c=net['GencodeBasicV12_NAME'].dropna().astype(object)
    c=jeff4.T[j].dropna().astype(object)
    genes_str='\n'.join(c)
    ENRICHR_URL = 'http://maayanlab.cloud/Enrichr/addList'
    query_string = '?userListId=%s&backgroundType=%s'

    description = 'Example gene list'
    payload = {
        'list': (None, genes_str),
        'description': (None, description)
    }

    response = requests.post(ENRICHR_URL, files=payload)
    if not response.ok:
        raise Exception('Error analyzing gene list')

    data = json.loads(response.text)

    user_list_id = data['userListId']
    gene_set_library = 'GO_Biological_Process_2018'
    ENRICHR_URL = 'http://maayanlab.cloud/Enrichr/enrich'
    response = requests.get(
        ENRICHR_URL + query_string % (user_list_id, gene_set_library)
     )
    if not response.ok:
        raise Exception('Error fetching enrichment results')

    data = json.loads(response.text);
    processBio=[]
    pvals=[]
    for i in range(1,10):
        processBio.append(data['GO_Biological_Process_2018'][i][1])
        pvals.append(data['GO_Biological_Process_2018'][i][2])
    plt.figure()
    y_pos = np.arange(len(processBio))
    plt.barh(y_pos, -np.log10(pvals))
    plt.yticks(ticks=y_pos,labels=processBio);
    plt.gca().invert_yaxis()  
    plt.xlabel('-log(p-value)')
#     plt.title(file.split('/')[3])
#     pd.DataFrame(processBio).to_csv('~/analyses/LTRC/bench/GEO/'+file.split('/')[3])
# #     except ValueError:
# #         pass

#     jeff=jeff.append(['~/analyses/LTRC/bench/GEO/'+file.split('/')[3]])
#     jeff=jeff.append(pd.DataFrame(processBio))
# jeff.to_csv('~/analyses/LTRC/bench/overall_enrich.txt',sep='\t')
